In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Feb 12 09:47:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%%capture
!pip install datasets transformers huggingface_hub torchaudio librosa jiwer
!pip install accelerate -U
!pip install nvidia-smi
!pip install transformers[torch]
!pip install wandb
!pip install matplotlib seaborn
!pip install evaluate
import os
import re
import wandb
import torch
import numpy as np
import json
from evaluate import load
from datasets import load_dataset, Audio
from transformers import EarlyStoppingCallback
from huggingface_hub import HfFolder


In [ ]:
import wandb
import torch
import numpy as np
import json
from datasets import load_dataset,  Audio
from transformers import (     Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor,
    Wav2Vec2ForCTC, TrainingArguments, Trainer
)
from dataclasses import dataclass
from typing import Any, Dict, List, Union
# from dotenv import load_dotenv
from transformers import EarlyStoppingCallback
from huggingface_hub import login

In [ ]:
HF_API_TOKEN = os.getenv("HF_TOKEN")  # Secure environment variable used instead of hardcoded tokenSXwdwQiQBwpzvZdjPObRIxFxMSQWspDVFJ"
WANDB_API_KEY = "796635845bbcb935e270354f88bafb37436ad1c4"
# Load datasets
# Login to WandB and Hugging Face
wandb.login(key=WANDB_API_KEY)
login(HF_API_TOKEN)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kyagabajonah (asr-africa-research-team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Install required packages if needed:
# !pip install jiwer pandas evaluate

import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import jiwer
import unicodedata
import re
import evaluate

########################################
# 1. Helper: Normalize Amharic
########################################

def normalize_amharic(text):
    """
    Normalize Amharic text.
    This function uses Unicode NFC normalization and lowercases the text.
    Adjust further as needed.
    """
    return unicodedata.normalize("NFC", str(text)).lower()

########################################
# 2. Custom Alignment (Word-Level)
########################################

def align_tokens(ref_tokens, hyp_tokens):
    """
    Perform a dynamic programming alignment (Levenshtein) on two lists of tokens.
    Returns a list of tuples (ref_word, hyp_word, operation),
    where operation is one of: "hit", "sub", "ins", "del".
    """
    n = len(ref_tokens)
    m = len(hyp_tokens)
    dp = [[(0, []) for _ in range(m + 1)] for _ in range(n + 1)]

    # Initialize first column (all deletions)
    for i in range(1, n + 1):
        cost_prev, align_prev = dp[i - 1][0]
        dp[i][0] = (cost_prev + 1, align_prev + [(ref_tokens[i - 1], "", "del")])

    # Initialize first row (all insertions)
    for j in range(1, m + 1):
        cost_prev, align_prev = dp[0][j - 1]
        dp[0][j] = (cost_prev + 1, align_prev + [("", hyp_tokens[j - 1], "ins")])

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            ref_word = ref_tokens[i - 1]
            hyp_word = hyp_tokens[j - 1]
            if ref_word == hyp_word:
                cost_prev, align_prev = dp[i - 1][j - 1]
                dp[i][j] = (cost_prev, align_prev + [(ref_word, hyp_word, "hit")])
            else:
                # substitution
                sub_cost, sub_align = dp[i - 1][j - 1]
                sub_cost += 1
                sub_align = sub_align + [(ref_word, hyp_word, "sub")]
                # insertion
                ins_cost, ins_align = dp[i][j - 1]
                ins_cost += 1
                ins_align = ins_align + [("", hyp_word, "ins")]
                # deletion
                del_cost, del_align = dp[i - 1][j]
                del_cost += 1
                del_align = del_align + [(ref_word, "", "del")]
                if sub_cost <= ins_cost and sub_cost <= del_cost:
                    dp[i][j] = (sub_cost, sub_align)
                elif ins_cost <= del_cost:
                    dp[i][j] = (ins_cost, ins_align)
                else:
                    dp[i][j] = (del_cost, del_align)

    _, alignment_list = dp[n][m]
    return alignment_list

########################################
# 3. Levenshtein Edit Distance (Char-Level)
########################################

def levenshtein_distance(a, b):
    """
    Compute the edit (Levenshtein) distance between two strings a and b.
    """
    A, B = len(a), len(b)
    dp = np.zeros((A + 1, B + 1), dtype=int)
    for i in range(A + 1):
        dp[i][0] = i
    for j in range(B + 1):
        dp[0][j] = j
    for i in range(1, A + 1):
        for j in range(1, B + 1):
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[i][j] = min(
                dp[i-1][j] + 1,
                dp[i][j-1] + 1,
                dp[i-1][j-1] + cost
            )
    return dp[A][B]

########################################
# 4. Morphological "Root" (Placeholder)
########################################

def get_morphological_root(amharic_word):
    """
    A naive placeholder to remove some common Amharic suffixes.
    Replace or extend this function with a real morphological analyzer if needed.
    """
    suffixes = ["ም", "ማ", "ች", "ቹ", "ው", "ዎች", "ነ", "ዋ", "ቷ", "ሗ", "ሉ"]
    root = amharic_word
    for sfx in suffixes:
        if root.endswith(sfx) and len(root) > len(sfx):
            root = root[:-len(sfx)]
    return root

########################################
# 5. Compute Metrics and Alignment per Example
########################################

def compute_individual_metrics(hypothesis, reference):
    """
    Compute WER, CER, error counts, and perform word-level alignment.
    Returns a tuple: (wer, cer, error_counts, alignment)
    """
    ref_norm = normalize_amharic(reference)
    hyp_norm = normalize_amharic(hypothesis)

    measures = jiwer.compute_measures(ref_norm, hyp_norm)
    wer_val = measures.get("wer", 0.0)
    sub = measures.get("substitutions", 0)
    ins = measures.get("insertions", 0)
    dele = measures.get("deletions", 0)
    total_words = len(ref_norm.split())
    errors = {
        "substitutions": sub,
        "insertions": ins,
        "deletions": dele,
        "total_words": total_words
    }

    cer_val = jiwer.cer(ref_norm, hyp_norm)

    ref_tokens = ref_norm.split()
    hyp_tokens = hyp_norm.split()
    alignment = align_tokens(ref_tokens, hyp_tokens)

    return wer_val, cer_val, errors, alignment

########################################
# 6. Comprehensive Substitution Analysis
########################################

def analyze_substitutions_deeper(substitution_list, top_n=10):
    pair_counts = defaultdict(int)
    context_counts = defaultdict(int)
    root_counts = defaultdict(int)
    char_pair_counts = defaultdict(int)

    for item in substitution_list:
        r_word = item["ref_word"]
        h_word = item["hyp_word"]
        ctx = tuple(item["preceding_context"])
        r_root = item["ref_root"]
        h_root = item["hyp_root"]
        for (r_char, h_char) in item["char_mismatches"]:
            char_pair_counts[(r_char, h_char)] += 1
        pair_counts[(r_word, h_word)] += 1
        context_counts[ctx] += 1
        root_counts[(r_root, h_root)] += 1

    sorted_pairs = sorted(pair_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Substitution Word Patterns:")
    for (r, h), cnt in sorted_pairs[:top_n]:
        print(f"  '{r}' -> '{h}': {cnt} times")

    sorted_contexts = sorted(context_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Preceding Contexts:")
    for ctx, cnt in sorted_contexts[:top_n]:
        print(f"  Context {ctx}: {cnt} times")

    sorted_roots = sorted(root_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Substitution Root Patterns:")
    for (r_root, h_root), cnt in sorted_roots[:top_n]:
        if not r_root and not h_root:
            continue
        print(f"  '{r_root}' -> '{h_root}': {cnt} times")

    sorted_char_pairs = sorted(char_pair_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Character Mismatch Pairs:")
    for (r_c, h_c), cnt in sorted_char_pairs[:top_n]:
        print(f"  '{r_c}' -> '{h_c}': {cnt} times")

########################################
# 7. Analyze CSV File
########################################

def analyze_csv(csv_file, top_n=10):
    """
    Reads a CSV file with columns ["Prediction", "Reference"], computes per-row metrics,
    and prints overall as well as detailed substitution analyses.
    """
    df = pd.read_csv(csv_file)
    results_rows = []
    substitution_list = []
    total_subs = total_ins = total_del = total_wds = 0
    all_predictions = []
    all_references = []

    for idx, row in df.iterrows():
        pred = str(row["Prediction"])
        ref = str(row["Reference"])
        wer_val, cer_val, errors, alignment = compute_individual_metrics(pred, ref)
        total_subs += errors["substitutions"]
        total_ins += errors["insertions"]
        total_del += errors["deletions"]
        total_wds += errors["total_words"]
        all_predictions.append(pred)
        all_references.append(ref)
        results_rows.append({
            "Prediction": pred,
            "Reference": ref,
            "WER": wer_val,
            "CER": cer_val,
            "Substitutions": errors["substitutions"],
            "Insertions": errors["insertions"],
            "Deletions": errors["deletions"]
        })

        ref_tokens = [a[0] for a in alignment]
        hyp_tokens = [a[1] for a in alignment]
        ops = [a[2] for a in alignment]
        for i, op in enumerate(ops):
            if op == "sub":
                start_idx = max(0, i - 2)
                preceding_context = ref_tokens[start_idx:i]
                ref_word = ref_tokens[i]
                hyp_word = hyp_tokens[i]
                next_word = ref_tokens[i+1] if (i+1) < len(ref_tokens) else None
                char_dist = levenshtein_distance(ref_word, hyp_word)
                char_mismatches = []
                max_len = max(len(ref_word), len(hyp_word))
                for c in range(max_len):
                    r_c = ref_word[c] if c < len(ref_word) else ""
                    h_c = hyp_word[c] if c < len(hyp_word) else ""
                    if r_c != h_c:
                        char_mismatches.append((r_c, h_c))
                ref_root = get_morphological_root(ref_word)
                hyp_root = get_morphological_root(hyp_word)
                substitution_list.append({
                    "row_idx": idx,
                    "preceding_context": preceding_context,
                    "ref_word": ref_word,
                    "hyp_word": hyp_word,
                    "next_word": next_word,
                    "char_distance": char_dist,
                    "char_mismatches": char_mismatches,
                    "ref_root": ref_root,
                    "hyp_root": hyp_root
                })

    results_df = pd.DataFrame(results_rows)
    overall_wer = evaluate.load("wer").compute(predictions=all_predictions, references=all_references)
    overall_cer = evaluate.load("cer").compute(predictions=all_predictions, references=all_references)

    print("\n=== Overall Metrics ===")
    print(f"Overall WER: {overall_wer}")
    print(f"Overall CER: {overall_cer}")
    print(f"Total Substitutions: {total_subs}")
    print(f"Total Insertions: {total_ins}")
    print(f"Total Deletions: {total_del}")
    print(f"Total Words: {total_wds}")
    if total_wds > 0:
        print(f"Substitution Rate: {total_subs/total_wds:.2f}")
        print(f"Insertion Rate: {total_ins/total_wds:.2f}")
        print(f"Deletion Rate: {total_del/total_wds:.2f}")

    print("\n=== Sample Row-Level Results ===")
    print(results_df.head(5))

    print("\n=== Deeper Substitution Analysis ===")
    analyze_substitutions_deeper(substitution_list, top_n=top_n)

########################################
# 8. Analyze All CSV Files in a Folder
########################################

# Assuming your CSV files (for each model and dataset) are stored in the folder "csv_results"
csv_folder = "csv_results"
# Now, instead of filtering only combined CSVs, include every CSV file.
csv_files = [os.path.join(csv_folder, f) for f in os.listdir(csv_folder) if f.endswith(".csv")]

for csv_file in csv_files:
    print("\n====================================")
    print(f"Analyzing results from: {csv_file}")
    analyze_csv(csv_file, top_n=10)



Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-200hrs-1_FLEURS_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.9681056777188893
Total Substitutions: 49606
Total Insertions: 0
Total Deletions: 257604
Total Words: 307210
Substitution Rate: 0.16
Insertion Rate: 0.00
Deletion Rate: 0.84

=== Sample Row-Level Results ===
                                          Prediction  \
0  በመድርማ ማር ዳል ማዳላ ዳድራ ከንቶጉም በዳሞ ቱዳሙ ከቸጉርው እይድድ ተ...   
1  ጥላልው ሪፓድ ሊ ልኔ ማዳ አልፈለ ከንዶሚያጉ ወና መዱከበጋንኛው ኤንለ ኮ...   
2  ደጀማው የለስጋካ አተው ምስኮሎፎፎንት ጋንዲ የሪፈርቶ ን ስጤ እንደማላው ...   
3  ዲላ ማኮማ በድ ይኛ የቡዋ ስዎው የደ ላማርን አመሪጋ ለሌበንሚሊዮንቱ ንት...   
4  ሲቡዎች ማሞን ፊሌ አ ቤዶ እ ማቻል መት በኖ አሄበ ረቶዎ ኛው ፋልግ ማለ...   

                                           Reference  WER       CER  \
0  ['m', 'a', 'd', 'h', '', 'm', 'a', 't', 'h', '...  1.0  0.965812   
1  ['o', 's', 'e', 'c', 'h', 'i', 'w', '', 'r', '...  1.0  0.966777   
2  ['p', 'i', 'c', 'h', 'a', '', 'm', 'a', 'c', '...  1.0  0.977827   
3  ['m', '

In [ ]:
import re
import pandas as pd

# --- Example input text ---
# Replace the contents of the raw_text variable with your complete printed output.
raw_text = """

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-200hrs-1_FLEURS_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.9681056777188893
Total Substitutions: 49606
Total Insertions: 0
Total Deletions: 257604
Total Words: 307210
Substitution Rate: 0.16
Insertion Rate: 0.00
Deletion Rate: 0.84

=== Sample Row-Level Results ===
                                          Prediction  \
0  በመድርማ ማር ዳል ማዳላ ዳድራ ከንቶጉም በዳሞ ቱዳሙ ከቸጉርው እይድድ ተ...
1  ጥላልው ሪፓድ ሊ ልኔ ማዳ አልፈለ ከንዶሚያጉ ወና መዱከበጋንኛው ኤንለ ኮ...
2  ደጀማው የለስጋካ አተው ምስኮሎፎፎንት ጋንዲ የሪፈርቶ ን ስጤ እንደማላው ...
3  ዲላ ማኮማ በድ ይኛ የቡዋ ስዎው የደ ላማርን አመሪጋ ለሌበንሚሊዮንቱ ንት...
4  ሲቡዎች ማሞን ፊሌ አ ቤዶ እ ማቻል መት በኖ አሄበ ረቶዎ ኛው ፋልግ ማለ...

                                           Reference  WER       CER  \
0  ['m', 'a', 'd', 'h', '', 'm', 'a', 't', 'h', '...  1.0  0.965812
1  ['o', 's', 'e', 'c', 'h', 'i', 'w', '', 'r', '...  1.0  0.966777
2  ['p', 'i', 'c', 'h', 'a', '', 'm', 'a', 'c', '...  1.0  0.977827
3  ['m', 'i', 'd', 'i', 'l', 'a', '', 'm', 'o', '...  1.0  0.971831
4  ['s', 'i', 'b', 'u', 'o', 'c', 'h', 'e', '', '...  1.0  0.967066

   Substitutions  Insertions  Deletions
0             24           0        123
1             20           0        105
2             11           0         82
3             17           0        101
4             23           0        115

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''',' -> 'ን': 86 times
  ''o']' -> 'ነው': 74 times
  ''',' -> 'ደ': 61 times
  ''o',' -> 'ን': 57 times
  ''',' -> 'ማር': 54 times
  ''a',' -> 'ን': 51 times
  ''',' -> 'ማ': 48 times
  ''a',' -> 'ማር': 46 times
  ''',' -> 'ነው': 44 times
  ''n',' -> 'ን': 42 times

Top 10 Preceding Contexts:
  Context ("'',", "'m',"): 1929 times
  Context ("'o',", "'',"): 1791 times
  Context ("'m',", "'a',"): 1699 times
  Context ("'e',", "'',"): 1555 times
  Context ("'',", "'k',"): 1030 times
  Context ("'a',", "'',"): 1022 times
  Context ("'i',", "'',"): 886 times
  Context ("'a',", "'r',"): 720 times
  Context ("'',", "'n',"): 678 times
  Context ("'c',", "'h',"): 661 times

Top 10 Substitution Root Patterns:
  ''',' -> 'ን': 99 times
  ''o']' -> 'ነ': 79 times
  ''',' -> 'ደ': 74 times
  ''',' -> 'ነ': 72 times
  ''o',' -> 'ን': 63 times
  ''',' -> 'ማር': 58 times
  ''',' -> 'ማ': 58 times
  ''a',' -> 'ን': 53 times
  ''a',' -> 'ማር': 49 times
  ''e',' -> 'ን': 49 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 30002 times
  ''' -> '': 18802 times
  ''' -> 'ማ': 5787 times
  ''' -> 'ን': 5264 times
  ''' -> 'የ': 3439 times
  ''' -> 'በ': 2779 times
  ''' -> 'አ': 2611 times
  ''' -> 'ው': 2268 times
  '' -> 'ን': 2055 times
  ''' -> 'እ': 2047 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-2_COMMON_VOICE_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8395498717465454
Total Substitutions: 3820
Total Insertions: 0
Total Deletions: 18760
Total Words: 22580
Substitution Rate: 0.17
Insertion Rate: 0.00
Deletion Rate: 0.83

=== Sample Row-Level Results ===
                                          Prediction  \
0   በጉን ለጉን እንዴ ጤንኛሆኗ ረጅ ማ መት መኖር እንጅላለን የሚለውም ይሚነሳል
1     በተፈጠረ በት ጊዜጀምሮ ለእራት ቢሌና  መታት የኖኦሮ የጠፈራ ካላት ናቸው
2  ኤሚቴ በመጨረሻም የጃንጃዊት ተከታይተብሎ ሊገለጽሚችሎ የፈጥሮ ደራሻይሎች ...
3          ሌሎች ደግሞ ከመሪ ነዋ ካርቶም እየሄሹ ወደሌሎችን ዛቶች ይይደዋል
4  የዩ ክሬን ባለስልጣናት መሪ ውእናናታከታሄርቻቸውን ከገዳሙ ለማሶወጣ ጥራት...

                                           Reference  WER       CER  \
0  ['ጎ', 'ን', '', 'ለ', 'ጎ', 'ን', '', 'እ', 'ን', 'ዴ...  1.0  0.834043
1  ['ከ', 'ተ', 'ፈ', 'ጠ', 'ረ', 'በ', 'ት', '', 'ጊ', '...  1.0  0.827434
2  ['ሄ', 'ም', 'ቲ', '', 'በ', 'መ', 'ጨ', 'ረ', 'ሻ', '...  1.0  0.848592
3  ['ሌ', 'ሎ', 'ች', '', 'ደ', 'ግ', 'ሞ', '', 'ከ', 'መ...  1.0  0.828877
4  ['የ', 'ዩ', 'ክ', 'ሬ', 'ን', '', 'ባ', 'ለ', 'ስ', '...  1.0  0.816479

   Substitutions  Insertions  Deletions
0             11           0         38
1             10           0         37
2              9           0         50
3              9           0         30
4             10           0         45

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 39 times
  ''ር']' -> 'ነበር': 19 times
  ''ል']' -> 'ብለዋል': 7 times
  ''ው']' -> 'ናቸው': 6 times
  ''ም']' -> 'አይደለም': 6 times
  ''ረ',' -> 'ነው': 4 times
  ''',' -> 'ጋር': 4 times
  ''ለ']' -> 'አለ': 3 times
  ''ው']' -> 'ው': 3 times
  ''ሉ']' -> 'ሉ': 3 times

Top 10 Preceding Contexts:
  Context ("'',", "'አ',"): 123 times
  Context ("'',", "'ነ',"): 118 times
  Context ("'ት',", "'',"): 81 times
  Context ("'ው',", "'',"): 70 times
  Context ("'',", "'ይ',"): 68 times
  Context ("'ን',", "'',"): 64 times
  Context ("'',", "'ተ',"): 59 times
  Context ("'ም',", "'',"): 52 times
  Context ("'ር',", "'',"): 40 times
  Context ("'ል',", "'',"): 34 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 40 times
  ''ር']' -> 'ነበር': 19 times
  ''ል']' -> 'ብለዋል': 7 times
  ''ው']' -> 'ናቸ': 6 times
  ''ም']' -> 'አይደለ': 6 times
  ''ረ',' -> 'ነ': 4 times
  ''',' -> 'ጋር': 4 times
  ''ለ']' -> 'አለ': 3 times
  ''ው']' -> 'ው': 3 times
  ''ሉ']' -> 'ሉ': 3 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 986 times
  ''' -> 'በ': 478 times
  ''' -> 'የ': 472 times
  ''' -> '': 421 times
  '' -> 'ን': 390 times
  '' -> 'ው': 302 times
  ''' -> 'አ': 296 times
  '' -> 'ት': 262 times
  ''' -> 'ን': 246 times
  ''' -> 'እ': 234 times

====================================
Analyzing results from: csv_results/w2v-bert-2.0-amharic-dataset-amh-20hrs-2_AMHARIC_ALFA_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8024800744891167
Total Substitutions: 52830
Total Insertions: 0
Total Deletions: 201160
Total Words: 253990
Substitution Rate: 0.21
Insertion Rate: 0.00
Deletion Rate: 0.79

=== Sample Row-Level Results ===
                                          Prediction  \
0  ምግብ ከሱፐር ማርኬት መግዛት በተለምዶ ርካሹ ራስን የመመገቢያ ዘሌ ነው ...
1  ጃስ ሰይንሲግሪፐን በማኮብኮቢያ ሜዳ ላይ በዘጠኝ ተኩል ኤኤም አካባቢያዊ ...
2  ሰው አልባው የጨረቃ ሳተላይት ቻንድሪያን አንድ የሞን ኢምፓክት ፒሮቡል ወ...
3  ቅፅበታዊ የፅሁፍ አስተርጓሚ መተግበሪያዎች ሙሉ የፅሁፍ ክፍሎችን ከአንድ ...
4   ሶቪየት ዩኒየን ትልቅ ክብር የሶቪየት ዩኒየን ጀግና ለስራው ስራ ሁሉ ተሰጠው

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.802000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.799007
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.797980
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.792541
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.800000

   Substitutions  Insertions  Deletions
0             22           0         82
1             25           0         60
2             33           0         91
3             17           0         72
4             11           0         38

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 440 times
  ''',' -> 'እና': 272 times
  ''ር']' -> 'ነበር': 200 times
  ''',' -> 'ላይ': 150 times
  ''',' -> 'ውስጥ': 124 times
  ''ው']' -> 'ናቸው': 108 times
  ''ሉ']' -> 'ይችላሉ': 99 times
  ''ል']' -> 'ይችላል': 92 times
  ''',' -> 'ጊዜ': 86 times
  ''',' -> 'ወደ': 83 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 1142 times
  Context ("'ን',", "'',"): 1047 times
  Context ("'',", "'አ',"): 981 times
  Context ("'',", "'ነ',"): 884 times
  Context ("'',", "'ይ',"): 803 times
  Context ("'',", "'የ',"): 773 times
  Context ("'',", "'እ',"): 644 times
  Context ("'ው',", "'',"): 600 times
  Context ("'ች',", "'',"): 597 times
  Context ("'',", "'በ',"): 569 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 440 times
  ''',' -> 'እና': 292 times
  ''ር']' -> 'ነበር': 200 times
  ''',' -> 'ላይ': 150 times
  ''',' -> 'ውስጥ': 124 times
  ''ው']' -> 'ናቸ': 108 times
  ''ሉ']' -> 'ይችላ': 99 times
  ''ል']' -> 'ይችላል': 92 times
  ''',' -> 'ጊዜ': 87 times
  ''',' -> 'ወደ': 83 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 18824 times
  ''' -> '': 7055 times
  ''' -> 'የ': 6391 times
  ''' -> 'በ': 5570 times
  ''' -> 'አ': 4202 times
  ''' -> 'እ': 3787 times
  '' -> 'ን': 3201 times
  ''' -> 'መ': 3094 times
  '' -> 'ት': 2932 times
  ''' -> 'ን': 2736 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-2_combined_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8923170729825605
Total Substitutions: 122600
Total Insertions: 0
Total Deletions: 685473
Total Words: 808073
Substitution Rate: 0.15
Insertion Rate: 0.00
Deletion Rate: 0.85

=== Sample Row-Level Results ===
                                          Prediction  \
0  በእግብ ከስሱፕርማርኪት መግዘት በተለምዱ እርካሹራስን የመመገብ ያዘደ ምግ...
1  ሸያህሱ  ሳቲናዩንሴ ግሪፕን በማኮብኮጋ ሜዳለ  በዘጠን ተኩል ኤየኤ አከባ...
2  በሰሰ አልባው የጨረቃሳ ትላይትቻንድሪያን አንየሙን ኢንፓርትፒሮቡን ይም ኤ...
3  ቅስፈታዊ የጽፋስተርጓሚ መተግበሪዎዎች ሙሩ የጽፍ ክፍሎችን ከዱ ንቃ ዋሊዳ...
4      ሶቤት ውኒ የን ትልቅ ህብር የሶቤት ዩኒን ጀግና ለስራው ስራሁን ተሰጠው

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.838000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.826303
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.850168
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.850816
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.834043

   Substitutions  Insertions  Deletions
0             16           0         88
1             22           0         63
2             23           0        101
3             14           0         75
4             11           0         38

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 405 times
  ''o']' -> 'ነው': 154 times
  ''ር']' -> 'ነበር': 126 times
  ''',' -> 'ነው': 92 times
  ''ው']' -> 'ናቸው': 84 times
  ''ል']' -> 'ይችላል': 59 times
  ''',' -> 'ማር': 57 times
  ''',' -> 'ነ': 48 times
  ''',' -> 'የ': 47 times
  ''ር']' -> 'ነበል': 41 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 1862 times
  Context ("'',", "'አ',"): 1797 times
  Context ("'ን',", "'',"): 1612 times
  Context ("'',", "'m',"): 1572 times
  Context ("'',", "'ነ',"): 1537 times
  Context ("'o',", "'',"): 1408 times
  Context ("'m',", "'a',"): 1369 times
  Context ("'',", "'ይ',"): 1248 times
  Context ("'e',", "'',"): 1214 times
  Context ("'',", "'የ',"): 1131 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 417 times
  ''o']' -> 'ነ': 161 times
  ''',' -> 'ነ': 146 times
  ''ር']' -> 'ነበር': 136 times
  ''ው']' -> 'ናቸ': 89 times
  ''',' -> 'የ': 68 times
  ''',' -> 'ማር': 60 times
  ''ል']' -> 'ይችላል': 59 times
  ''ሉ']' -> 'ይችላ': 48 times
  ''',' -> 'ብዙ': 47 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 38885 times
  ''' -> '': 18438 times
  ''' -> 'የ': 15650 times
  '' -> 'ን': 13077 times
  ''' -> 'በ': 10812 times
  '' -> 'ው': 8729 times
  ''' -> 'አ': 8201 times
  ''' -> 'ን': 8046 times
  ''' -> 'መ': 7888 times
  '' -> 'ት': 7806 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-10hrs-3_AMHARIC_ALFA_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8144788220752694
Total Substitutions: 52651
Total Insertions: 0
Total Deletions: 201339
Total Words: 253990
Substitution Rate: 0.21
Insertion Rate: 0.00
Deletion Rate: 0.79

=== Sample Row-Level Results ===
                                          Prediction  \
0  ምግብ ከሱፐር ማርኬት መግዛት በተለምዶ ርካሹ እራስን የመመገብያዘሬ ነው ...
1  ጃስ ሰቲናይንሲ ግሪፐን በማኮብፎ በያ ሜዳ ለይ በዘጠን ተኩል የኤም አከባ...
2  ሰውአልባው የጨረቃሳ ትላይት ቻንድሪያን አንድ የሞን ኢምፓት ፒሮቡን ወይም...
3  ቅስፈታዊ የፅሁፍ አስተርጓሚ መተግብራዎች ሙሩ የፅሁፍ ክፍሎችን ከአድጓንቀ...
4       ሶቪየት ዩኔን ትልቅክብር  የሶቪየት ዩኔን ጀግና ለሰራው ስራሁሉ ተሰጠ

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.802000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.811414
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.808081
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.827506
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.821277

   Substitutions  Insertions  Deletions
0             22           0         82
1             26           0         59
2             31           0         93
3             14           0         75
4              9           0         40

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 433 times
  ''ር']' -> 'ነበር': 196 times
  ''',' -> 'ላይ': 129 times
  ''',' -> 'ውስጥ': 119 times
  ''',' -> 'እና': 119 times
  ''ው']' -> 'ናቸው': 104 times
  ''ሉ']' -> 'ይችላሉ': 86 times
  ''',' -> 'ወይም': 83 times
  ''ል']' -> 'ይችላል': 81 times
  ''',' -> 'ወደ': 77 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 1149 times
  Context ("'ን',", "'',"): 1035 times
  Context ("'',", "'አ',"): 982 times
  Context ("'',", "'ነ',"): 881 times
  Context ("'',", "'ይ',"): 806 times
  Context ("'',", "'የ',"): 786 times
  Context ("'',", "'እ',"): 627 times
  Context ("'ች',", "'',"): 595 times
  Context ("'ው',", "'',"): 595 times
  Context ("'',", "'በ',"): 574 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 433 times
  ''ር']' -> 'ነበር': 196 times
  ''',' -> 'እና': 145 times
  ''',' -> 'ላይ': 130 times
  ''',' -> 'ውስጥ': 119 times
  ''ው']' -> 'ናቸ': 105 times
  ''',' -> 'ወይ': 89 times
  ''ሉ']' -> 'ይችላ': 86 times
  ''ል']' -> 'ይችላል': 81 times
  ''',' -> 'ወደ': 77 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 18587 times
  ''' -> '': 7213 times
  ''' -> 'የ': 6149 times
  ''' -> 'በ': 5667 times
  ''' -> 'አ': 4184 times
  '' -> 'ን': 3328 times
  ''' -> 'መ': 3304 times
  '' -> 'ት': 3067 times
  ''' -> 'እ': 2947 times
  ''' -> 'ስ': 2902 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-200hrs-1_COMMON_VOICE_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.7932407211613604
Total Substitutions: 4573
Total Insertions: 0
Total Deletions: 18007
Total Words: 22580
Substitution Rate: 0.20
Insertion Rate: 0.00
Deletion Rate: 0.80

=== Sample Row-Level Results ===
                                          Prediction  \
0  ጎን ለጎን እንዴት ጤነኛ ሆነን ረዥም አመት መኖር እንችላለን የሚለውም ይነሳል
1    ከተፈጠረበት ጊዜ ጀምሮ ለአራት ቢሊዮን አመታት የኖሩ የጠፈር አካላት ናቸው
2  ሄምቲ በመጨረሻም የጃንጃዊድ ተከታይ ተብሎ ሊገለፅ የሚችለው የፈጥኖ ደራሽ...
3            ሌሎች ደግሞ ከመዲናዋ ካርቱም እየሸሹ ወደ ሌሎች ግዛቶች ሄዷል
4  የዩክሬን ባለስልጣናት መሪውና ተከታዮቻቸውን ከገዳሙ ለማስወጣት ጥረት ሲያ...

                                           Reference  WER       CER  \
0  ['ጎ', 'ን', '', 'ለ', 'ጎ', 'ን', '', 'እ', 'ን', 'ዴ...  1.0  0.791489
1  ['ከ', 'ተ', 'ፈ', 'ጠ', 'ረ', 'በ', 'ት', '', 'ጊ', '...  1.0  0.792035
2  ['ሄ', 'ም', 'ቲ', '', 'በ', 'መ', 'ጨ', 'ረ', 'ሻ', '...  1.0  0.792254
3  ['ሌ', 'ሎ', 'ች', '', 'ደ', 'ግ', 'ሞ', '', 'ከ', 'መ...  1.0  0.791444
4  ['የ', 'ዩ', 'ክ', 'ሬ', 'ን', '', 'ባ', 'ለ', 'ስ', '...  1.0  0.794007

   Substitutions  Insertions  Deletions
0             11           0         38
1             10           0         37
2             12           0         47
3              9           0         30
4              9           0         46

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 59 times
  ''ር']' -> 'ነበር': 23 times
  ''',' -> 'እና': 18 times
  ''ው']' -> 'ናቸው': 10 times
  ''ም']' -> 'አይደለም': 9 times
  ''ል']' -> 'ብለዋል': 7 times
  ''',' -> 'ጋር': 7 times
  ''ሉ']' -> 'ያስረዳሉ': 6 times
  ''',' -> 'በኋላ': 6 times
  ''',' -> 'ወደ': 5 times

Top 10 Preceding Contexts:
  Context ("'',", "'አ',"): 142 times
  Context ("'',", "'ነ',"): 122 times
  Context ("'ት',", "'',"): 100 times
  Context ("'ው',", "'',"): 84 times
  Context ("'ን',", "'',"): 80 times
  Context ("'ም',", "'',"): 71 times
  Context ("'',", "'ይ',"): 70 times
  Context ("'',", "'ተ',"): 62 times
  Context ("'ር',", "'',"): 46 times
  Context ("'ል',", "'',"): 37 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 59 times
  ''ር']' -> 'ነበር': 23 times
  ''',' -> 'እና': 18 times
  ''ው']' -> 'ናቸ': 10 times
  ''ም']' -> 'አይደለ': 9 times
  ''ል']' -> 'ብለዋል': 7 times
  ''',' -> 'ጋር': 7 times
  ''ሉ']' -> 'ያስረዳ': 6 times
  ''',' -> 'በኋላ': 6 times
  ''',' -> 'ወደ': 5 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 1452 times
  ''' -> '': 544 times
  ''' -> 'የ': 533 times
  ''' -> 'አ': 464 times
  ''' -> 'በ': 412 times
  ''' -> 'እ': 349 times
  '' -> 'ው': 308 times
  '' -> 'ን': 270 times
  ''' -> 'መ': 252 times
  '' -> 'ት': 245 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-100hrs_AMHARIC_DATASET_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8112077529264369
Total Substitutions: 44485
Total Insertions: 0
Total Deletions: 179808
Total Words: 224293
Substitution Rate: 0.20
Insertion Rate: 0.00
Deletion Rate: 0.80

=== Sample Row-Level Results ===
                                          Prediction  \
0  የኢንተርኔት አገልግሎትንም በተመለከተ በክልሎች በዞኖችና በአዲስ አበባ በ...
1  በዚህም አነስተኛ መስኖችን በማስዋፋት አገሪቱን በተደጋጋሚ ከሚያጠቃት ድር...
2  ይህ እንዳይሆን በህይወታቸው መስዋዕትነት ያስገኙትን የኢትዮጵያዊነት ክብር...
3  የመጀመሪያው ነጥብ በተቃዋሚ ሀይሎች ሰፈር በጋራ አቋም ላይ በጋራ ለመቆም...
4  አቅም በሚፈቅረው መሰረት አስቀድሞው መዘጋጀት ስለሚፈልጉ የሚያጠራጥር አይ...

                                           Reference  WER       CER  \
0  ['የ', 'ኢ', 'ን', 'ተ', 'ር', 'ኔ', 'ት', '', 'አ', '...  1.0  0.794050
1  ['በ', 'ዚ', 'ህ', 'ም', '', 'አ', 'ነ', 'ስ', 'ተ', '...  1.0  0.798489
2  ['ይ', 'ህ', '', 'እ', 'ን', 'ዳ', 'ይ', 'ሆ', 'ን', '...  1.0  0.797938
3  ['የ', 'መ', 'ጀ', 'መ', 'ሪ', 'ያ', 'ው', '', 'ነ', '...  1.0  0.794479
4  ['አ', 'ቅ', 'ም', '', 'በ', 'ሚ', 'ፈ', 'ቅ', 'ደ', '...  1.0  0.795580

   Substitutions  Insertions  Deletions
0             14           0         76
1             14           0         68
2             17           0         83
3             15           0         53
4             14           0         61

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 153 times
  ''',' -> 'ነው': 77 times
  ''',' -> 'ላይ': 73 times
  ''',' -> 'እና': 56 times
  ''ር']' -> 'ነበር': 54 times
  ''',' -> 'ግን': 52 times
  ''ው']' -> 'ናቸው': 45 times
  ''',' -> 'ወደ': 37 times
  ''',' -> 'ና': 32 times
  ''ዉ']' -> 'ነው': 32 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 1082 times
  Context ("'',", "'አ',"): 980 times
  Context ("'ን',", "'',"): 865 times
  Context ("'',", "'የ',"): 801 times
  Context ("'',", "'ነ',"): 618 times
  Context ("'',", "'እ',"): 614 times
  Context ("'ው',", "'',"): 605 times
  Context ("'ም',", "'',"): 498 times
  Context ("'',", "'መ',"): 497 times
  Context ("'',", "'ይ',"): 447 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 154 times
  ''',' -> 'ነ': 78 times
  ''',' -> 'ላይ': 74 times
  ''',' -> 'እና': 58 times
  ''ር']' -> 'ነበር': 55 times
  ''',' -> 'ግን': 52 times
  ''ው']' -> 'ናቸ': 45 times
  ''',' -> 'ወደ': 37 times
  ''',' -> 'ና': 33 times
  ''ዉ']' -> 'ነ': 32 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 15273 times
  ''' -> '': 5937 times
  ''' -> 'የ': 5355 times
  ''' -> 'በ': 4135 times
  ''' -> 'አ': 4129 times
  ''' -> 'መ': 3039 times
  '' -> 'ት': 2813 times
  '' -> 'ን': 2699 times
  ''' -> 'እ': 2688 times
  '' -> 'ው': 2645 times

====================================
Analyzing results from: csv_results/w2v-bert-2.0-amharic-dataset-amh-20hrs-2_FLEURS_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.9710308886332828
Total Substitutions: 45264
Total Insertions: 0
Total Deletions: 261946
Total Words: 307210
Substitution Rate: 0.15
Insertion Rate: 0.00
Deletion Rate: 0.85

=== Sample Row-Level Results ===
                                          Prediction  \
0  መዝማት ማግ ከኛ መሯክ ኦካዘ ዶኝ ኮሙ ቱዩሞ ፖጉሬ እይ ዴ ደኛቸዋቱ ከኛ...
1  ወሰቺው ሪፖርትን ዴልኒ ማኮ አልፈች ከኮድሚያ ገን መኒቲር ኤጎን ና ኦንጌ...
2  ቢጀማች የለውኝ ሰው ከከ አቶው ምት ኮሎ ፎቶንስ ኬንዋዲ ሪፎቶን እስቲ እ...
3  ሚዲላ መኮማ በንዴ ቲና ይየህኛ የፒየንኢው ሶኤዶላ ማር አሜርካ ሌቬን ሚሊ...
4  ሲቡዎቼ ማሞን ፒሌ ጀቤዶጋ ማቻልድራ አችቺ ግኔ አሄኛ እኛባ ንጌኛው አድጌ...

                                           Reference  WER       CER  \
0  ['m', 'a', 'd', 'h', '', 'm', 'a', 't', 'h', '...  1.0  0.964387
1  ['o', 's', 'e', 'c', 'h', 'i', 'w', '', 'r', '...  1.0  0.966777
2  ['p', 'i', 'c', 'h', 'a', '', 'm', 'a', 'c', '...  1.0  0.973392
3  ['m', 'i', 'd', 'i', 'l', 'a', '', 'm', 'o', '...  1.0  0.964789
4  ['s', 'i', 'b', 'u', 'o', 'c', 'h', 'e', '', '...  1.0  0.971557

   Substitutions  Insertions  Deletions
0             25           0        122
1             21           0        104
2             13           0         80
3             21           0         97
4             20           0        118

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''o']' -> 'ነው': 107 times
  ''',' -> 'ማር': 76 times
  ''',' -> 'ነው': 71 times
  ''a',' -> 'ማር': 59 times
  ''o',' -> 'ማር': 44 times
  ''',' -> 'ነ': 43 times
  ''o',' -> 'ነው': 40 times
  ''e',' -> 'ማር': 40 times
  ''a',' -> 'ነው': 38 times
  ''',' -> 'ኮድ': 37 times

Top 10 Preceding Contexts:
  Context ("'',", "'m',"): 1776 times
  Context ("'o',", "'',"): 1610 times
  Context ("'m',", "'a',"): 1555 times
  Context ("'e',", "'',"): 1404 times
  Context ("'',", "'k',"): 912 times
  Context ("'a',", "'',"): 907 times
  Context ("'i',", "'',"): 787 times
  Context ("'a',", "'r',"): 660 times
  Context ("'',", "'n',"): 616 times
  Context ("'c',", "'h',"): 599 times

Top 10 Substitution Root Patterns:
  ''',' -> 'ነ': 116 times
  ''o']' -> 'ነ': 112 times
  ''',' -> 'ማር': 79 times
  ''o',' -> 'ነ': 63 times
  ''a',' -> 'ማር': 60 times
  ''a',' -> 'ነ': 53 times
  ''',' -> 'ማ': 53 times
  ''n',' -> 'ነ': 47 times
  ''e',' -> 'ነ': 46 times
  ''o',' -> 'ማር': 46 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 22832 times
  ''' -> '': 12333 times
  ''' -> 'ማ': 6993 times
  ''' -> 'ን': 3705 times
  ''' -> 'አ': 2622 times
  ''' -> 'የ': 2505 times
  '' -> 'ን': 2413 times
  ''' -> 'ኤ': 1972 times
  ''' -> 'ኮ': 1967 times
  ',' -> 'ን': 1898 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-50hrs-3_combined_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8803240720500716
Total Substitutions: 139511
Total Insertions: 0
Total Deletions: 668562
Total Words: 808073
Substitution Rate: 0.17
Insertion Rate: 0.00
Deletion Rate: 0.83

=== Sample Row-Level Results ===
                                          Prediction  \
0  ምግብ ከፓርማርኬት መግዛት በተለምዱ ርካሹ እራስን የመመገበያ ዘሬ ነው ም...
1  ጃስ ሰቲናይንሲ ግሪፐን በማኮብኮበያ ሜዳ ለይ ለዘጠኝ ተኩል ኤኤም አከባ ...
2  ሰው አልባው የረቃሳ ላይት ንድሪ አን የሞን ኢንፓት ፒሮዳን ወይ ኤምአራ ...
3  ቅፅበታዊ የፅፍ አስተርጓሚ መተግበሪያዎች ሙሉ የፅሁፍ ክፍሎችን ከአድ ቋን...
4           ሶቪዬት ዩኔን ትልቅ ብር የሶቬት ዩኔን ጀግ ለስራ ራ ሁሉ ተሰጠ

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.812000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.816377
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.835017
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.801865
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.851064

   Substitutions  Insertions  Deletions
0             21           0         83
1             26           0         59
2             30           0         94
3             17           0         72
4             11           0         38

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 521 times
  ''ር']' -> 'ነበር': 251 times
  ''',' -> 'ላይ': 206 times
  ''',' -> 'እና': 177 times
  ''ው']' -> 'ናቸው': 144 times
  ''',' -> 'ውስጥ': 131 times
  ''',' -> 'ወደ': 130 times
  ''',' -> 'ነው': 115 times
  ''',' -> 'እ': 108 times
  ''',' -> 'ር': 108 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 2203 times
  Context ("'',", "'አ',"): 2029 times
  Context ("'ን',", "'',"): 1899 times
  Context ("'',", "'m',"): 1642 times
  Context ("'',", "'ነ',"): 1599 times
  Context ("'',", "'የ',"): 1516 times
  Context ("'o',", "'',"): 1464 times
  Context ("'m',", "'a',"): 1440 times
  Context ("'',", "'ይ',"): 1304 times
  Context ("'e',", "'',"): 1284 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 598 times
  ''ር']' -> 'ነበር': 253 times
  ''',' -> 'ነ': 237 times
  ''',' -> 'ላይ': 207 times
  ''',' -> 'እና': 196 times
  ''',' -> 'እ': 154 times
  ''ው']' -> 'ናቸ': 150 times
  ''',' -> 'ወደ': 131 times
  ''',' -> 'ውስጥ': 131 times
  ''',' -> 'ር': 114 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 63093 times
  ''' -> '': 34459 times
  ''' -> 'የ': 15270 times
  ''' -> 'አ': 12389 times
  ''' -> 'እ': 10693 times
  ''' -> 'በ': 10681 times
  ''' -> 'መ': 7271 times
  ''' -> 'ን': 6719 times
  ''' -> 'ነ': 6621 times
  '' -> 'ን': 6523 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-100hrs_FLEURS_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.975365685134917
Total Substitutions: 38855
Total Insertions: 0
Total Deletions: 268355
Total Words: 307210
Substitution Rate: 0.13
Insertion Rate: 0.00
Deletion Rate: 0.87

=== Sample Row-Level Results ===
                                          Prediction  \
0  መዝማትመ ኮ መዖግኦካሎ ዶን ኮሞ ትዮሞርም  ተንድ ግንዶርም ከእኛን ቀረው...
1  ኦሰው ሪፖርትኒ ዴልኒማኮ አልፈቺ ከድሚን መንዲዲሬ አይ ጎንናው ኦንገ ኮድ...
2  ቢቻማች ለውኛነሰው ጋአታው ስኮሎፎታውንስ ጎንረ ዲሪፎተውን እስእንደሞላዋ ...
3  ሜዴላ ሞከማ በንደቲና ህን ኤፒንኢውሰው ኤዶላ ማር ሜሪርካ ሌቪን ሚሊዮን ...
4  ሲብዎ የመሞንፊ የቤዶ ጋር መቻል ድ መር ግኔ አሄኛ  ኛዋ አል መልሰው አ...

                                           Reference  WER       CER  \
0  ['m', 'a', 'd', 'h', '', 'm', 'a', 't', 'h', '...  1.0  0.977208
1  ['o', 's', 'e', 'c', 'h', 'i', 'w', '', 'r', '...  1.0  0.976744
2  ['p', 'i', 'c', 'h', 'a', '', 'm', 'a', 'c', '...  1.0  0.980044
3  ['m', 'i', 'd', 'i', 'l', 'a', '', 'm', 'o', '...  1.0  0.971831
4  ['s', 'i', 'b', 'u', 'o', 'c', 'h', 'e', '', '...  1.0  0.968563

   Substitutions  Insertions  Deletions
0             16           0        131
1             15           0        110
2             10           0         83
3             17           0        101
4             21           0        117

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''o']' -> 'ነው': 107 times
  ''',' -> 'ን': 64 times
  ''',' -> 'ማር': 61 times
  ''',' -> 'ነው': 56 times
  ''',' -> 'ር': 42 times
  ''a',' -> 'ማር': 36 times
  ''o',' -> 'ማር': 36 times
  ''n',' -> 'ማር': 33 times
  ''a',' -> 'ር': 28 times
  ''i',' -> 'ር': 28 times

Top 10 Preceding Contexts:
  Context ("'',", "'m',"): 1531 times
  Context ("'m',", "'a',"): 1362 times
  Context ("'o',", "'',"): 1343 times
  Context ("'e',", "'',"): 1168 times
  Context ("'a',", "'',"): 785 times
  Context ("'',", "'k',"): 735 times
  Context ("'i',", "'',"): 650 times
  Context ("'a',", "'r',"): 599 times
  Context ("'c',", "'h',"): 528 times
  Context ("'r',", "'',"): 517 times

Top 10 Substitution Root Patterns:
  ''o']' -> 'ነ': 110 times
  ''',' -> 'ነ': 86 times
  ''',' -> 'ን': 68 times
  ''',' -> 'ማር': 64 times
  ''',' -> 'ር': 42 times
  ''a',' -> 'ማር': 38 times
  ''o',' -> 'ነ': 36 times
  ''o',' -> 'ማር': 36 times
  ''',' -> 'እ': 35 times
  ''',' -> 'አ': 35 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 20384 times
  ''' -> '': 11890 times
  ''' -> 'ማ': 4471 times
  ''' -> 'ን': 3253 times
  ''' -> 'አ': 2507 times
  ''' -> 'የ': 2402 times
  ''' -> 'ወ': 2269 times
  '' -> 'ን': 1858 times
  ''' -> 'መ': 1847 times
  ''' -> 'ር': 1668 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-100hrs_AMHARIC_ALFA_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8049963771798394
Total Substitutions: 52582
Total Insertions: 0
Total Deletions: 201408
Total Words: 253990
Substitution Rate: 0.21
Insertion Rate: 0.00
Deletion Rate: 0.79

=== Sample Row-Level Results ===
                                          Prediction  \
0  ምግብ ከሱፐርማርኬት መግዛት በተለምዶ ርካሹ ራስን የመመገበ ዘዴ ነው ምግ...
1  ጃስሰይ ሪፐን በማኮብፖበ ሜዳ ላይ በኝ ተኩል ኤኤም አከባቢያዊ ሰአት በዜ...
2  ሰው አልባው የጨረቃ ተላይትቻንድሪያን አንድ የሞ ኢንፓክት ፒሮቡን ወይም ...
3  ቅፅበታዊ የፅሁፍ አስተርጓሚ መተግበሪያዎች ሙሉ የፅሁፍ ክፍሎችን ከአንድ ...
4     ሶውዬት ዩኒን ትልቅ ክብር የሶቬት ዩኒየን ጀግና ለስራው ስራ ሁሉ ተሰጠው

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.804000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.808933
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.816498
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.794872
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.821277

   Substitutions  Insertions  Deletions
0             21           0         83
1             23           0         62
2             28           0         96
3             17           0         72
4             11           0         38

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 432 times
  ''',' -> 'እና': 229 times
  ''ር']' -> 'ነበር': 192 times
  ''',' -> 'ላይ': 142 times
  ''ው']' -> 'ናቸው': 103 times
  ''ሉ']' -> 'ይችላሉ': 97 times
  ''',' -> 'ውስጥ': 94 times
  ''ል']' -> 'ይችላል': 92 times
  ''',' -> 'ጊዜ': 84 times
  ''',' -> 'ወደ': 77 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 1145 times
  Context ("'ን',", "'',"): 1044 times
  Context ("'',", "'አ',"): 978 times
  Context ("'',", "'ነ',"): 883 times
  Context ("'',", "'ይ',"): 808 times
  Context ("'',", "'የ',"): 776 times
  Context ("'',", "'እ',"): 648 times
  Context ("'ች',", "'',"): 597 times
  Context ("'ው',", "'',"): 596 times
  Context ("'',", "'በ',"): 568 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 435 times
  ''',' -> 'እና': 254 times
  ''ር']' -> 'ነበር': 194 times
  ''',' -> 'ላይ': 142 times
  ''ው']' -> 'ናቸ': 104 times
  ''ሉ']' -> 'ይችላ': 97 times
  ''',' -> 'ውስጥ': 94 times
  ''ል']' -> 'ይችላል': 92 times
  ''',' -> 'ጊዜ': 86 times
  ''',' -> 'ወደ': 77 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 19048 times
  ''' -> '': 7789 times
  ''' -> 'የ': 6207 times
  ''' -> 'በ': 5518 times
  ''' -> 'አ': 4111 times
  ''' -> 'እ': 3638 times
  '' -> 'ን': 3188 times
  ''' -> 'መ': 3159 times
  '' -> 'ት': 2929 times
  ''' -> 'ን': 2617 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-10hrs-3_AMHARIC_DATASET_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8178686865691558
Total Substitutions: 42095
Total Insertions: 0
Total Deletions: 182198
Total Words: 224293
Substitution Rate: 0.19
Insertion Rate: 0.00
Deletion Rate: 0.81

=== Sample Row-Level Results ===
                                          Prediction  \
0  የኢንተርኔት አገልግሎተንን በተመለከተ በክልሎች በዞኖችና በአዲስ አበባ በ...
1  በዚህም አነስተኛ ምስኖችን በማስዋፋት አገሪቱን በተደጋጋሚ ከሚያጠቃት ድር...
2  ይህ እንዳይሆን በህወታቸው መስዋእትነት ያስገኙትን የኢትዮጵያዊነት ክብር ...
3  የመጀመሪያው ነጥብ በተቃዋሚ ሀይሎች ሰፈር በጋራ አቋም ላይ በጋራ ለማቆም...
4  አቅም በሚፈቅረው መሰረት አስቀድሞ መዘጋጀት ስለሚ ፈልጉ የሚያጠራጥር አይ...

                                           Reference  WER       CER  \
0  ['የ', 'ኢ', 'ን', 'ተ', 'ር', 'ኔ', 'ት', '', 'አ', '...  1.0  0.798627
1  ['በ', 'ዚ', 'ህ', 'ም', '', 'አ', 'ነ', 'ስ', 'ተ', '...  1.0  0.801008
2  ['ይ', 'ህ', '', 'እ', 'ን', 'ዳ', 'ይ', 'ሆ', 'ን', '...  1.0  0.797938
3  ['የ', 'መ', 'ጀ', 'መ', 'ሪ', 'ያ', 'ው', '', 'ነ', '...  1.0  0.791411
4  ['አ', 'ቅ', 'ም', '', 'በ', 'ሚ', 'ፈ', 'ቅ', 'ደ', '...  1.0  0.792818

   Substitutions  Insertions  Deletions
0             14           0         76
1             14           0         68
2             17           0         83
3             15           0         53
4             15           0         60

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 155 times
  ''',' -> 'ነው': 66 times
  ''',' -> 'ላይ': 59 times
  ''ር']' -> 'ነበር': 48 times
  ''ው']' -> 'ናቸው': 46 times
  ''ዉ']' -> 'ነው': 43 times
  ''',' -> 'ወደ': 41 times
  ''',' -> 'ግን': 36 times
  ''',' -> 'ውስጥ': 33 times
  ''',' -> 'ደግሞ': 30 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 1002 times
  Context ("'',", "'አ',"): 932 times
  Context ("'ን',", "'',"): 820 times
  Context ("'',", "'የ',"): 749 times
  Context ("'',", "'ነ',"): 599 times
  Context ("'ው',", "'',"): 580 times
  Context ("'',", "'እ',"): 560 times
  Context ("'ም',", "'',"): 478 times
  Context ("'',", "'መ',"): 468 times
  Context ("'',", "'ይ',"): 442 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 159 times
  ''',' -> 'ነ': 74 times
  ''',' -> 'ላይ': 59 times
  ''ር']' -> 'ነበር': 51 times
  ''ው']' -> 'ናቸ': 47 times
  ''ዉ']' -> 'ነ': 46 times
  ''',' -> 'ወደ': 41 times
  ''',' -> 'ግን': 36 times
  ''',' -> 'ውስጥ': 33 times
  ''',' -> 'ደግሞ': 30 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 12959 times
  ''' -> 'የ': 5133 times
  ''' -> '': 4834 times
  ''' -> 'በ': 4082 times
  ''' -> 'አ': 3674 times
  '' -> 'ን': 3102 times
  ''' -> 'መ': 3018 times
  '' -> 'ት': 2977 times
  '' -> 'ው': 2862 times
  ''' -> 'እ': 2314 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-50hrs-3_AMHARIC_DATASET_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8268824458266304
Total Substitutions: 41793
Total Insertions: 0
Total Deletions: 182500
Total Words: 224293
Substitution Rate: 0.19
Insertion Rate: 0.00
Deletion Rate: 0.81

=== Sample Row-Level Results ===
                                          Prediction  \
0  የኢንተርኔት አገልግሎትን በተመለከተ በክልሎች በዞኖችና በአዲስ አበባ በተ...
1  በዚህም አነስተኛ መስኖችን በማሷዋፋት አገሪቱን በተደጋጋሚ ከሚያጠቃት ድር...
2  ይህ እንዳይሆን በህይወታቸው መስዋእትነት ያስገኙትን የኢትዮጵያዊነት ክብር...
3  የመጀመሪያ ነጥብ በተቃዋሚ ሀይሎች ሰፍር በጋራ አቋም ላይ በጋራ ለማቆም ...
4  አቅም በሚፈቅደው መሰረት ኤስቀድሞ መዘጋጀት ስለሚ ፈልጉ የሚያጠራጥር አይ...

                                           Reference  WER       CER  \
0  ['የ', 'ኢ', 'ን', 'ተ', 'ር', 'ኔ', 'ት', '', 'አ', '...  1.0  0.796339
1  ['በ', 'ዚ', 'ህ', 'ም', '', 'አ', 'ነ', 'ስ', 'ተ', '...  1.0  0.798489
2  ['ይ', 'ህ', '', 'እ', 'ን', 'ዳ', 'ይ', 'ሆ', 'ን', '...  1.0  0.802062
3  ['የ', 'መ', 'ጀ', 'መ', 'ሪ', 'ያ', 'ው', '', 'ነ', '...  1.0  0.797546
4  ['አ', 'ቅ', 'ም', '', 'በ', 'ሚ', 'ፈ', 'ቅ', 'ደ', '...  1.0  0.795580

   Substitutions  Insertions  Deletions
0             14           0         76
1             15           0         67
2             15           0         85
3             15           0         53
4             15           0         60

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 125 times
  ''',' -> 'ላይ': 71 times
  ''',' -> 'ነው': 57 times
  ''',' -> 'ወደ': 45 times
  ''ር']' -> 'ነበር': 44 times
  ''ው']' -> 'ናቸው': 36 times
  ''ዉ']' -> 'ነው': 36 times
  ''',' -> 'ውስጥ': 29 times
  ''',' -> 'ግን': 26 times
  ''',' -> 'መንግስት': 25 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 988 times
  Context ("'',", "'አ',"): 930 times
  Context ("'ን',", "'',"): 812 times
  Context ("'',", "'የ',"): 735 times
  Context ("'',", "'ነ',"): 600 times
  Context ("'ው',", "'',"): 569 times
  Context ("'',", "'እ',"): 562 times
  Context ("'ም',", "'',"): 479 times
  Context ("'',", "'መ',"): 462 times
  Context ("'',", "'ይ',"): 433 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 145 times
  ''',' -> 'ላይ': 72 times
  ''',' -> 'ነ': 69 times
  ''ር']' -> 'ነበር': 46 times
  ''',' -> 'ወደ': 45 times
  ''ው']' -> 'ናቸ': 39 times
  ''ዉ']' -> 'ነ': 37 times
  ''',' -> 'ውስጥ': 29 times
  ''',' -> 'እንደ': 26 times
  ''',' -> 'መንግስት': 26 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 14160 times
  ''' -> '': 5998 times
  ''' -> 'የ': 5185 times
  ''' -> 'በ': 3951 times
  ''' -> 'አ': 3746 times
  ''' -> 'መ': 3031 times
  '' -> 'ት': 2726 times
  '' -> 'ን': 2670 times
  '' -> 'ው': 2530 times
  ''' -> 'እ': 2454 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-50hrs-3_AMHARIC_ALFA_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.819494214962739
Total Substitutions: 51500
Total Insertions: 0
Total Deletions: 202490
Total Words: 253990
Substitution Rate: 0.20
Insertion Rate: 0.00
Deletion Rate: 0.80

=== Sample Row-Level Results ===
                                          Prediction  \
0  ምግብ ከፓርማርኬት መግዛት በተለምዱ ርካሹ እራስን የመመገበያ ዘሬ ነው ም...
1  ጃስ ሰቲናይንሲ ግሪፐን በማኮብኮበያ ሜዳ ለይ ለዘጠኝ ተኩል ኤኤም አከባ ...
2  ሰው አልባው የረቃሳ ላይት ንድሪ አን የሞን ኢንፓት ፒሮዳን ወይ ኤምአራ ...
3  ቅፅበታዊ የፅፍ አስተርጓሚ መተግበሪያዎች ሙሉ የፅሁፍ ክፍሎችን ከአድ ቋን...
4           ሶቪዬት ዩኔን ትልቅ ብር የሶቬት ዩኔን ጀግ ለስራ ራ ሁሉ ተሰጠ

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.812000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.816377
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.835017
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.801865
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.851064

   Substitutions  Insertions  Deletions
0             21           0         83
1             26           0         59
2             30           0         94
3             17           0         72
4             11           0         38

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 352 times
  ''ር']' -> 'ነበር': 184 times
  ''',' -> 'እና': 164 times
  ''',' -> 'ላይ': 126 times
  ''',' -> 'ውስጥ': 102 times
  ''ው']' -> 'ናቸው': 98 times
  ''ሉ']' -> 'ይችላሉ': 91 times
  ''ል']' -> 'ይችላል': 89 times
  ''',' -> 'ወደ': 77 times
  ''',' -> 'ጊዜ': 64 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 1118 times
  Context ("'ን',", "'',"): 1014 times
  Context ("'',", "'አ',"): 960 times
  Context ("'',", "'ነ',"): 879 times
  Context ("'',", "'ይ',"): 801 times
  Context ("'',", "'የ',"): 749 times
  Context ("'',", "'እ',"): 622 times
  Context ("'ው',", "'',"): 583 times
  Context ("'ች',", "'',"): 572 times
  Context ("'',", "'ተ',"): 560 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 403 times
  ''ር']' -> 'ነበር': 184 times
  ''',' -> 'እና': 181 times
  ''',' -> 'ላይ': 126 times
  ''',' -> 'ውስጥ': 102 times
  ''ው']' -> 'ናቸ': 101 times
  ''ሉ']' -> 'ይችላ': 92 times
  ''ል']' -> 'ይችላል': 89 times
  ''',' -> 'ወደ': 77 times
  ''',' -> 'ጊዜ': 65 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 19871 times
  ''' -> '': 8880 times
  ''' -> 'የ': 6435 times
  ''' -> 'በ': 5086 times
  ''' -> 'አ': 3856 times
  ''' -> 'እ': 3431 times
  ''' -> 'መ': 3090 times
  '' -> 'ን': 2728 times
  '' -> 'ት': 2646 times
  ''' -> 'ስ': 2481 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-2_AMHARIC_DATASET_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8414553231868694
Total Substitutions: 34634
Total Insertions: 0
Total Deletions: 189659
Total Words: 224293
Substitution Rate: 0.15
Insertion Rate: 0.00
Deletion Rate: 0.85

=== Sample Row-Level Results ===
                                          Prediction  \
0  የንተርኔት አገልግሎተንንበተመለከተ በከልሎች  በዞኖችና በአአዲስ አበባ በ...
1  በዝዚ ህም  አነስተኛ ምስኖችን በማሷፋት አገሪቱን በተደጋጋሚ ከሚያተቀት ...
2  ይህ እንዳይሆን በሄወታቸው መሶእትነት ያስገኙትንኢትዮጵያዊነት ከበር አንገ...
3  የምጅ መሪያው ነጸብ በተቃዋሚ ሀይሎች ሰፈር በጋራ አቋምላይበጋውራ ለማ ቆ...
4  አቅን በሚ ፈቅው መሰረት  ስቀድሞ መዘጋጀት ስለሚ ፈል[UNK]የሚያጠራአጠ...

                                           Reference  WER       CER  \
0  ['የ', 'ኢ', 'ን', 'ተ', 'ር', 'ኔ', 'ት', '', 'አ', '...  1.0  0.810069
1  ['በ', 'ዚ', 'ህ', 'ም', '', 'አ', 'ነ', 'ስ', 'ተ', '...  1.0  0.803526
2  ['ይ', 'ህ', '', 'እ', 'ን', 'ዳ', 'ይ', 'ሆ', 'ን', '...  1.0  0.826804
3  ['የ', 'መ', 'ጀ', 'መ', 'ሪ', 'ያ', 'ው', '', 'ነ', '...  1.0  0.800613
4  ['አ', 'ቅ', 'ም', '', 'በ', 'ሚ', 'ፈ', 'ቅ', 'ደ', '...  1.0  0.812155

   Substitutions  Insertions  Deletions
0             14           0         76
1             16           0         66
2             15           0         85
3             15           0         53
4             14           0         61

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 76 times
  ''ር']' -> 'ነበር': 31 times
  ''',' -> 'ነው': 30 times
  ''ም']' -> 'ም': 20 times
  ''ው']' -> 'ው': 19 times
  ''',' -> 'ላይ': 16 times
  ''ው']' -> 'ናቸው': 15 times
  ''',' -> 'መንግስት': 15 times
  ''ር']' -> 'ነበረ': 15 times
  ''ም']' -> 'የለም': 14 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 816 times
  Context ("'',", "'አ',"): 795 times
  Context ("'ን',", "'',"): 656 times
  Context ("'',", "'ነ',"): 565 times
  Context ("'',", "'የ',"): 544 times
  Context ("'ው',", "'',"): 479 times
  Context ("'',", "'እ',"): 452 times
  Context ("'ም',", "'',"): 405 times
  Context ("'',", "'ይ',"): 404 times
  Context ("'',", "'መ',"): 366 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 85 times
  ''',' -> 'ነ': 39 times
  ''ር']' -> 'ነበር': 35 times
  ''ም']' -> 'ም': 20 times
  ''ው']' -> 'ው': 19 times
  ''',' -> 'ላይ': 17 times
  ''ም']' -> 'የለ': 16 times
  ''',' -> 'መንግስት': 16 times
  ''ር']' -> 'ነበረ': 16 times
  ''ው']' -> 'ናቸ': 15 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 8956 times
  ''' -> 'የ': 4373 times
  '' -> 'ን': 4148 times
  ''' -> '': 3817 times
  ''' -> 'በ': 3440 times
  '' -> 'ት': 3439 times
  '' -> 'ው': 3103 times
  ''' -> 'አ': 2797 times
  ''' -> 'መ': 2539 times
  ''' -> 'እ': 2016 times

====================================
Analyzing results from: csv_results/w2v-bert-2.0-amharic-dataset-amh-20hrs-2_COMMON_VOICE_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8058270279215586
Total Substitutions: 4507
Total Insertions: 0
Total Deletions: 18073
Total Words: 22580
Substitution Rate: 0.20
Insertion Rate: 0.00
Deletion Rate: 0.80

=== Sample Row-Level Results ===
                                          Prediction  \
0  ጎን ለ[UNK]ን እንዴት ጤነኛ ሆና ረዥም አመት መኖር እንችላለን የሚለው...
1    ከተፈጠረበት ጊዜ ጀምሮ ለአራት ቢሊዮን አመታት የኖሩ የጠፈር አካላት ናቸው
2  ሄሚቲ በመጨረሻም የጃንጃዊት ተፈታይ ተብሎ ሊገለፅ የሚችለው የፈጥሮ ደረሻ...
3           ሌሎች ደግሞ ከመዲናዋ ካርቱም እየሸሹ ወደ ሌሎች ግዛቶች ሄደዋል
4  የዩክሬን ባለስልጣናት መሪውና ተከታዮቻቸውን ከገዳሙ ለማስወጣት ጥረት ሲያ...

                                           Reference  WER       CER  \
0  ['ጎ', 'ን', '', 'ለ', 'ጎ', 'ን', '', 'እ', 'ን', 'ዴ...  1.0  0.804255
1  ['ከ', 'ተ', 'ፈ', 'ጠ', 'ረ', 'በ', 'ት', '', 'ጊ', '...  1.0  0.792035
2  ['ሄ', 'ም', 'ቲ', '', 'በ', 'መ', 'ጨ', 'ረ', 'ሻ', '...  1.0  0.827465
3  ['ሌ', 'ሎ', 'ች', '', 'ደ', 'ግ', 'ሞ', '', 'ከ', 'መ...  1.0  0.796791
4  ['የ', 'ዩ', 'ክ', 'ሬ', 'ን', '', 'ባ', 'ለ', 'ስ', '...  1.0  0.794007

   Substitutions  Insertions  Deletions
0             11           0         38
1             10           0         37
2             12           0         47
3              9           0         30
4              9           0         46

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 55 times
  ''ር']' -> 'ነበር': 23 times
  ''',' -> 'እና': 17 times
  ''ው']' -> 'ናቸው': 10 times
  ''ም']' -> 'አይደለም': 9 times
  ''ል']' -> 'ብለዋል': 7 times
  ''',' -> 'ጋር': 7 times
  ''',' -> 'ወደ': 6 times
  ''ሉ']' -> 'ያስረዳሉ': 6 times
  ''ሉ']' -> 'ይላሉ': 5 times

Top 10 Preceding Contexts:
  Context ("'',", "'አ',"): 142 times
  Context ("'',", "'ነ',"): 122 times
  Context ("'ት',", "'',"): 96 times
  Context ("'ው',", "'',"): 83 times
  Context ("'ን',", "'',"): 78 times
  Context ("'ም',", "'',"): 70 times
  Context ("'',", "'ይ',"): 70 times
  Context ("'',", "'ተ',"): 63 times
  Context ("'ር',", "'',"): 46 times
  Context ("'',", "'የ',"): 36 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 55 times
  ''ር']' -> 'ነበር': 23 times
  ''',' -> 'እና': 17 times
  ''ው']' -> 'ናቸ': 10 times
  ''ም']' -> 'አይደለ': 9 times
  ''ል']' -> 'ብለዋል': 7 times
  ''',' -> 'ጋር': 7 times
  ''',' -> 'ወደ': 6 times
  ''ሉ']' -> 'ያስረዳ': 6 times
  ''ሉ']' -> 'ይላ': 5 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 1454 times
  ''' -> '': 553 times
  ''' -> 'የ': 526 times
  ''' -> 'አ': 418 times
  ''' -> 'በ': 414 times
  ''' -> 'እ': 322 times
  '' -> 'ው': 286 times
  '' -> 'ን': 263 times
  '' -> 'ት': 261 times
  ''' -> 'መ': 260 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-200hrs-1_AMHARIC_ALFA_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8047004862119727
Total Substitutions: 51533
Total Insertions: 0
Total Deletions: 202457
Total Words: 253990
Substitution Rate: 0.20
Insertion Rate: 0.00
Deletion Rate: 0.80

=== Sample Row-Level Results ===
                                          Prediction  \
0  ምግብ ከሶፐር ማርኬት መግዛት በተለምዶ ርከሹ እራስን የመመገቢያ ዘዴ ነው...
1  ጅስ ሳናይንሲ ግሪፐን የሞኮብኮበሜዳ ላይ በ ተኩል ኤኤም አከባቢያዊ ሰአት...
2  ሰው አልባው የጨረቃሳ ተላይት ቻንድሪርያን አንድ የሞ ኢምፓክት ፕሮቡን ኤ...
3  ቅፅበታዊ የፅሁፍ አስተርጓሚ መተግበሪያዎች ሙሉ የፅሁፍ ክፍሎችን ከአንድ ...
4  የሶቪየት ዩኒየን ትልቅ ክብር የሶቪየት ዩኒየን ጀግና ለሰራው ስራ ሁሉ ተሰጠው

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.802000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.818859
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.808081
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.792541
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.791489

   Substitutions  Insertions  Deletions
0             22           0         82
1             23           0         62
2             28           0         96
3             17           0         72
4             11           0         38

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 435 times
  ''',' -> 'እና': 238 times
  ''ር']' -> 'ነበር': 199 times
  ''',' -> 'ላይ': 135 times
  ''',' -> 'ውስጥ': 109 times
  ''ው']' -> 'ናቸው': 107 times
  ''ሉ']' -> 'ይችላሉ': 100 times
  ''ል']' -> 'ይችላል': 90 times
  ''',' -> 'ጊዜ': 79 times
  ''',' -> 'ወደ': 69 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 1124 times
  Context ("'ን',", "'',"): 1017 times
  Context ("'',", "'አ',"): 962 times
  Context ("'',", "'ነ',"): 886 times
  Context ("'',", "'ይ',"): 805 times
  Context ("'',", "'የ',"): 742 times
  Context ("'',", "'እ',"): 629 times
  Context ("'ች',", "'',"): 586 times
  Context ("'ው',", "'',"): 578 times
  Context ("'',", "'ተ',"): 560 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 436 times
  ''',' -> 'እና': 268 times
  ''ር']' -> 'ነበር': 199 times
  ''',' -> 'ላይ': 136 times
  ''',' -> 'ውስጥ': 109 times
  ''ው']' -> 'ናቸ': 107 times
  ''ሉ']' -> 'ይችላ': 100 times
  ''ል']' -> 'ይችላል': 90 times
  ''',' -> 'ጊዜ': 81 times
  ''',' -> 'ወደ': 69 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 18267 times
  ''' -> '': 7380 times
  ''' -> 'የ': 6374 times
  ''' -> 'በ': 5727 times
  ''' -> 'አ': 3882 times
  ''' -> 'እ': 3634 times
  '' -> 'ን': 3235 times
  ''' -> 'መ': 2997 times
  '' -> 'ት': 2921 times
  ''' -> 'ን': 2650 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-50hrs-3_COMMON_VOICE_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8266877511055539
Total Substitutions: 4364
Total Insertions: 0
Total Deletions: 18216
Total Words: 22580
Substitution Rate: 0.19
Insertion Rate: 0.00
Deletion Rate: 0.81

=== Sample Row-Level Results ===
                                          Prediction  \
0       ጎን ለጎን እንዴጤነኛ ሆና በዚማ እመት መዳር እንላለን የሚልም ይነሣል
1      ከተፈጠረብት ጊዜ ጀምሮ ለአራት ቢሌና መታት የኖሩ የጠፈር አካላት ናቸው
2  የሚቴ በመጨረሻ የጃንጃ ተከታይ ተብሎ ሊገልፅ የሚችሉ የፈጥሮ ደራሻራይች ...
3           ሌሎች ደግሞ ከመሪንዋ አካርቱም እየሹ ወደ ሌሎ ችግዛቶች ሄደዋል
4  የኢዩክሬን ቨለስልጣናት መሪውንና ተከታየሪቻቸውን ከገዳሙ ለማስወጣ ጥ ጥሬ...

                                           Reference  WER       CER  \
0  ['ጎ', 'ን', '', 'ለ', 'ጎ', 'ን', '', 'እ', 'ን', 'ዴ...  1.0  0.846809
1  ['ከ', 'ተ', 'ፈ', 'ጠ', 'ረ', 'በ', 'ት', '', 'ጊ', '...  1.0  0.814159
2  ['ሄ', 'ም', 'ቲ', '', 'በ', 'መ', 'ጨ', 'ረ', 'ሻ', '...  1.0  0.841549
3  ['ሌ', 'ሎ', 'ች', '', 'ደ', 'ግ', 'ሞ', '', 'ከ', 'መ...  1.0  0.812834
4  ['የ', 'ዩ', 'ክ', 'ሬ', 'ን', '', 'ባ', 'ለ', 'ስ', '...  1.0  0.808989

   Substitutions  Insertions  Deletions
0             10           0         39
1             10           0         37
2             11           0         48
3              9           0         30
4             10           0         45

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 44 times
  ''ር']' -> 'ነበር': 23 times
  ''ው']' -> 'ናቸው': 10 times
  ''ም']' -> 'አይደለም': 8 times
  ''ል']' -> 'ብለዋል': 6 times
  ''ው']' -> 'ነ': 6 times
  ''ሉ']' -> 'ይላሉ': 5 times
  ''',' -> 'ደግሞ': 5 times
  ''ሉ']' -> 'ያስረዳሉ': 5 times
  ''',' -> 'ወደ': 4 times

Top 10 Preceding Contexts:
  Context ("'',", "'አ',"): 139 times
  Context ("'',", "'ነ',"): 120 times
  Context ("'ት',", "'',"): 97 times
  Context ("'ው',", "'',"): 79 times
  Context ("'ን',", "'',"): 73 times
  Context ("'',", "'ይ',"): 70 times
  Context ("'ም',", "'',"): 66 times
  Context ("'',", "'ተ',"): 58 times
  Context ("'ር',", "'',"): 44 times
  Context ("'ል',", "'',"): 38 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 50 times
  ''ር']' -> 'ነበር': 23 times
  ''ው']' -> 'ናቸ': 10 times
  ''ም']' -> 'አይደለ': 8 times
  ''ል']' -> 'ብለዋል': 6 times
  ''',' -> 'ሆ': 5 times
  ''ሉ']' -> 'ይላ': 5 times
  ''',' -> 'ሰ': 5 times
  ''',' -> 'ደግሞ': 5 times
  ''ሉ']' -> 'ያስረዳ': 5 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 1546 times
  ''' -> '': 714 times
  ''' -> 'የ': 560 times
  ''' -> 'አ': 395 times
  ''' -> 'በ': 388 times
  ''' -> 'እ': 299 times
  ''' -> 'መ': 259 times
  '' -> 'ን': 243 times
  '' -> 'ው': 233 times
  ''' -> 'ነ': 231 times

====================================
Analyzing results from: csv_results/w2v-bert-2.0-amharic-dataset-amh-20hrs-2_AMHARIC_DATASET_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8095554929535699
Total Substitutions: 43808
Total Insertions: 0
Total Deletions: 180485
Total Words: 224293
Substitution Rate: 0.20
Insertion Rate: 0.00
Deletion Rate: 0.80

=== Sample Row-Level Results ===
                                          Prediction  \
0  የኢንተርኔት አገልግሎትንም በተመለከተ በክልሎች በዞኖችና በአዲስ አበባ በ...
1  በዚህም አነስተኛ ምስኖችን በማስፋፋት አገሪቱን በተደጋጋሚ ከሚያጠቃት ድር...
2  ይህ እንዳይሆን በህይወታቸው መስዋእትነት ያስገኙትን የኢትዮጵያዊነት ክብር...
3  የመጀመሪያው ነጥብ በተቃዋሚ ሀይሎች ሰፈር በጋራ አቋም ላይ በጋራ ለማቆም...
4  አቅም በሚፈቅደው መሰረት አስቀድሞ መዘጋጀት ስለሚ ፈልጉ የሚያጠራጥር አይ...

                                           Reference  WER       CER  \
0  ['የ', 'ኢ', 'ን', 'ተ', 'ር', 'ኔ', 'ት', '', 'አ', '...  1.0  0.794050
1  ['በ', 'ዚ', 'ህ', 'ም', '', 'አ', 'ነ', 'ስ', 'ተ', '...  1.0  0.801008
2  ['ይ', 'ህ', '', 'እ', 'ን', 'ዳ', 'ይ', 'ሆ', 'ን', '...  1.0  0.795876
3  ['የ', 'መ', 'ጀ', 'መ', 'ሪ', 'ያ', 'ው', '', 'ነ', '...  1.0  0.791411
4  ['አ', 'ቅ', 'ም', '', 'በ', 'ሚ', 'ፈ', 'ቅ', 'ደ', '...  1.0  0.790055

   Substitutions  Insertions  Deletions
0             14           0         76
1             14           0         68
2             16           0         84
3             15           0         53
4             15           0         60

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 164 times
  ''',' -> 'ነው': 71 times
  ''',' -> 'ላይ': 70 times
  ''ር']' -> 'ነበር': 54 times
  ''ው']' -> 'ናቸው': 48 times
  ''ዉ']' -> 'ነው': 48 times
  ''',' -> 'ግን': 47 times
  ''',' -> 'ወደ': 42 times
  ''',' -> 'እንደ': 40 times
  ''',' -> 'ደግሞ': 36 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 1048 times
  Context ("'',", "'አ',"): 967 times
  Context ("'ን',", "'',"): 858 times
  Context ("'',", "'የ',"): 785 times
  Context ("'',", "'ነ',"): 608 times
  Context ("'ው',", "'',"): 608 times
  Context ("'',", "'እ',"): 594 times
  Context ("'ም',", "'',"): 492 times
  Context ("'',", "'መ',"): 487 times
  Context ("'',", "'ይ',"): 448 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 166 times
  ''',' -> 'ነ': 76 times
  ''',' -> 'ላይ': 71 times
  ''ር']' -> 'ነበር': 54 times
  ''ው']' -> 'ናቸ': 49 times
  ''ዉ']' -> 'ነ': 48 times
  ''',' -> 'ግን': 47 times
  ''',' -> 'ወደ': 42 times
  ''',' -> 'እንደ': 42 times
  ''',' -> 'ደግሞ': 36 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 13690 times
  ''' -> 'የ': 5463 times
  ''' -> '': 4639 times
  ''' -> 'አ': 4374 times
  ''' -> 'በ': 4140 times
  ''' -> 'መ': 3001 times
  '' -> 'ት': 2952 times
  '' -> 'ን': 2900 times
  '' -> 'ው': 2887 times
  ''' -> 'እ': 2543 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-2_FLEURS_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.9746983777270618
Total Substitutions: 39824
Total Insertions: 0
Total Deletions: 267386
Total Words: 307210
Substitution Rate: 0.13
Insertion Rate: 0.00
Deletion Rate: 0.87

=== Sample Row-Level Results ===
                                          Prediction  \
0  መንድማት ማግ  ከም ኦክ ኦካዘው   ዶም  ኮኑ ዩ ሙፖጉሮ እዩኢዴንደገነች...
1  ኦሰጅውሪፖድኒ ዴልን ማኮ አልፎችከኮድሚያሙን መኒዲር የይ[UNK]ን ነው ኦ...
2  ቤቻማቻ የለኝሰ ቀአቸውም ስ  ኮሎ ፎቶን ኬንደወዲ እሪፎተው እንስዴንደ ሞ...
3  ሜድ ላሞ ኮማ በንደቴና ሂኛ የፒን ኢውስዋ ኤደላማር አመርጋለቨን የሚሊንቱ...
4  በኤሴቦች የማሞንበፒ ለየጀቤዶጋማቻ ልረ መችየግንአሄኛታው ጊኛው ዋድጊ ማለ...

                                           Reference  WER       CER  \
0  ['m', 'a', 'd', 'h', '', 'm', 'a', 't', 'h', '...  1.0  0.968661
1  ['o', 's', 'e', 'c', 'h', 'i', 'w', '', 'r', '...  1.0  0.970100
2  ['p', 'i', 'c', 'h', 'a', '', 'm', 'a', 'c', '...  1.0  0.968958
3  ['m', 'i', 'd', 'i', 'l', 'a', '', 'm', 'o', '...  1.0  0.966549
4  ['s', 'i', 'b', 'u', 'o', 'c', 'h', 'e', '', '...  1.0  0.976048

   Substitutions  Insertions  Deletions
0             17           0        130
1             16           0        109
2             12           0         81
3             18           0        100
4             16           0        122

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''o']' -> 'ነው': 154 times
  ''',' -> 'ማር': 56 times
  ''',' -> 'ነው': 43 times
  ''',' -> 'ነ': 38 times
  ''',' -> 'የ': 30 times
  ''o',' -> 'ነው': 29 times
  ''a']' -> 'ነው': 27 times
  ''n']' -> 'ነው': 27 times
  ''a',' -> 'ማር': 27 times
  ''a',' -> 'ነው': 26 times

Top 10 Preceding Contexts:
  Context ("'',", "'m',"): 1572 times
  Context ("'o',", "'',"): 1408 times
  Context ("'m',", "'a',"): 1369 times
  Context ("'e',", "'',"): 1214 times
  Context ("'a',", "'',"): 801 times
  Context ("'',", "'k',"): 765 times
  Context ("'i',", "'',"): 680 times
  Context ("'a',", "'r',"): 593 times
  Context ("'c',", "'h',"): 547 times
  Context ("'r',", "'',"): 523 times

Top 10 Substitution Root Patterns:
  ''o']' -> 'ነ': 161 times
  ''',' -> 'ነ': 83 times
  ''',' -> 'ማር': 59 times
  ''o',' -> 'ነ': 47 times
  ''',' -> 'የ': 44 times
  ''e',' -> 'ነ': 35 times
  ''a',' -> 'ነ': 35 times
  ''o',' -> 'የ': 34 times
  ''',' -> 'ማ': 33 times
  ''e']' -> 'ነ': 32 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 16431 times
  ''' -> '': 9027 times
  ''' -> 'የ': 5392 times
  ''' -> 'ማ': 5013 times
  '' -> 'ን': 3771 times
  ''' -> 'ን': 3386 times
  '' -> 'ው': 2526 times
  ''' -> 'መ': 2135 times
  ',' -> 'ን': 1885 times
  ''' -> 'አ': 1801 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-5hrs-2_AMHARIC_ALFA_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8248169738140592
Total Substitutions: 49346
Total Insertions: 0
Total Deletions: 204644
Total Words: 253990
Substitution Rate: 0.19
Insertion Rate: 0.00
Deletion Rate: 0.81

=== Sample Row-Level Results ===
                                          Prediction  \
0  ምግብ ከሱፐር ማርኬት መግዛት በተለምዱ እርካሹ ራስን የመመገበያ ዘዴ ነው...
1  ጃስሰቲናይንሲ ግሪፐን በመኮብፎበሜዳለይ በዘጠንተኩል ኤኤም አከባቢያዊ ሰአ...
2  ሰው አልባው የጨረቃሳ ተላይት ቻንድሪያን አንድ የሙን ኢንፓት ፒሮቡን ወይ...
3  ቅስፈታዊ የጹፍ አስተርጓሚ መተግበራዎች ሙሉ የጾህፍ ክፍሎችን ከአንድ ጓን...
4         ሶዌት ዩኔን ትልቅ ህብር የሶቬት ኔን ጀግና ለሰራው ስራሁሉ ተሰጠው

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.808000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.828784
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.816498
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.832168
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.842553

   Substitutions  Insertions  Deletions
0             22           0         82
1             20           0         65
2             29           0         95
3             15           0         74
4             10           0         39

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 406 times
  ''ር']' -> 'ነበር': 186 times
  ''ው']' -> 'ናቸው': 102 times
  ''',' -> 'ውስጥ': 85 times
  ''',' -> 'ላይ': 80 times
  ''ሉ']' -> 'ይችላሉ': 79 times
  ''ል']' -> 'ይችላል': 68 times
  ''',' -> 'ወደ': 64 times
  ''',' -> 'እና': 63 times
  ''',' -> 'ጊዜ': 55 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 1078 times
  Context ("'ን',", "'',"): 991 times
  Context ("'',", "'አ',"): 944 times
  Context ("'',", "'ነ',"): 872 times
  Context ("'',", "'ይ',"): 791 times
  Context ("'',", "'የ',"): 701 times
  Context ("'',", "'እ',"): 584 times
  Context ("'ው',", "'',"): 559 times
  Context ("'',", "'ተ',"): 545 times
  Context ("'ች',", "'',"): 543 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 407 times
  ''ር']' -> 'ነበር': 186 times
  ''ው']' -> 'ናቸ': 102 times
  ''',' -> 'ውስጥ': 86 times
  ''',' -> 'ላይ': 80 times
  ''ሉ']' -> 'ይችላ': 79 times
  ''',' -> 'እና': 76 times
  ''ል']' -> 'ይችላል': 68 times
  ''',' -> 'ወደ': 64 times
  ''',' -> 'ጊዜ': 57 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 15689 times
  ''' -> '': 5962 times
  ''' -> 'በ': 5445 times
  ''' -> 'የ': 5178 times
  ''' -> 'አ': 3944 times
  '' -> 'ን': 3746 times
  '' -> 'ት': 3027 times
  ''' -> 'መ': 2910 times
  ''' -> 'ስ': 2838 times
  ''' -> 'እ': 2765 times

====================================
Analyzing results from: csv_results/w2v-bert-2.0-amharic-dataset-amh-20hrs-2_combined_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8687139933517088
Total Substitutions: 146409
Total Insertions: 0
Total Deletions: 661664
Total Words: 808073
Substitution Rate: 0.18
Insertion Rate: 0.00
Deletion Rate: 0.82

=== Sample Row-Level Results ===
                                          Prediction  \
0  ምግብ ከሱፐር ማርኬት መግዛት በተለምዶ ርካሹ ራስን የመመገቢያ ዘሌ ነው ...
1  ጃስ ሰይንሲግሪፐን በማኮብኮቢያ ሜዳ ላይ በዘጠኝ ተኩል ኤኤም አካባቢያዊ ...
2  ሰው አልባው የጨረቃ ሳተላይት ቻንድሪያን አንድ የሞን ኢምፓክት ፒሮቡል ወ...
3  ቅፅበታዊ የፅሁፍ አስተርጓሚ መተግበሪያዎች ሙሉ የፅሁፍ ክፍሎችን ከአንድ ...
4   ሶቪየት ዩኒየን ትልቅ ክብር የሶቪየት ዩኒየን ጀግና ለስራው ስራ ሁሉ ተሰጠው

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.802000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.799007
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.797980
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.792541
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.800000

   Substitutions  Insertions  Deletions
0             22           0         82
1             25           0         60
2             33           0         91
3             17           0         72
4             11           0         38

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 659 times
  ''',' -> 'እና': 300 times
  ''ር']' -> 'ነበር': 277 times
  ''',' -> 'ላይ': 230 times
  ''',' -> 'ነው': 196 times
  ''ው']' -> 'ናቸው': 166 times
  ''',' -> 'ውስጥ': 164 times
  ''',' -> 'ወደ': 136 times
  ''o']' -> 'ነው': 107 times
  ''',' -> 'ጊዜ': 106 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 2286 times
  Context ("'',", "'አ',"): 2090 times
  Context ("'ን',", "'',"): 1983 times
  Context ("'',", "'m',"): 1776 times
  Context ("'',", "'ነ',"): 1614 times
  Context ("'o',", "'',"): 1610 times
  Context ("'',", "'የ',"): 1594 times
  Context ("'m',", "'a',"): 1555 times
  Context ("'e',", "'',"): 1404 times
  Context ("'',", "'ይ',"): 1321 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 661 times
  ''',' -> 'እና': 321 times
  ''ር']' -> 'ነበር': 277 times
  ''',' -> 'ነ': 246 times
  ''',' -> 'ላይ': 231 times
  ''ው']' -> 'ናቸ': 167 times
  ''',' -> 'ውስጥ': 164 times
  ''',' -> 'ወደ': 136 times
  ''',' -> 'ጊዜ': 112 times
  ''o']' -> 'ነ': 112 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 56800 times
  ''' -> '': 24580 times
  ''' -> 'የ': 14885 times
  ''' -> 'አ': 11616 times
  ''' -> 'በ': 11184 times
  ''' -> 'ማ': 9746 times
  '' -> 'ን': 8777 times
  ''' -> 'ን': 8727 times
  ''' -> 'እ': 8249 times
  ''' -> 'መ': 7785 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-10hrs-3_COMMON_VOICE_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8150943743162115
Total Substitutions: 4363
Total Insertions: 0
Total Deletions: 18217
Total Words: 22580
Substitution Rate: 0.19
Insertion Rate: 0.00
Deletion Rate: 0.81

=== Sample Row-Level Results ===
                                          Prediction  \
0      ጎንለቁን እንዴት ጤንኛ ሆነ ረዥማመት መኖር እንችላለን የሚለውም ይነሳል
1    ከተፈጠረብት ጊዜ ጀምሮ ለአራት ቢሊዮን መታት የየኖሩ የጠፈር አካላት ናቸው
2  ሄሚቴ በመጨረሻም የጃንጃዊት ተከታይ ተብሎ ሊገለጽ የሚችሎው የፈጥሮ ደራሻ...
3           ሌሎች ደግሞ ከመሪ ኑሀካርቱም እየሸሹ ወደ ሌሎችግ ዛቶች ሄደዋል
4  የዩክሬን ባለስልጣናት መሪውና ተከታየያቻቸውን ከገዳሙ ለማስወጣት ጥሬ ሲያ...

                                           Reference  WER       CER  \
0  ['ጎ', 'ን', '', 'ለ', 'ጎ', 'ን', '', 'እ', 'ን', 'ዴ...  1.0  0.821277
1  ['ከ', 'ተ', 'ፈ', 'ጠ', 'ረ', 'በ', 'ት', '', 'ጊ', '...  1.0  0.800885
2  ['ሄ', 'ም', 'ቲ', '', 'በ', 'መ', 'ጨ', 'ረ', 'ሻ', '...  1.0  0.820423
3  ['ሌ', 'ሎ', 'ች', '', 'ደ', 'ግ', 'ሞ', '', 'ከ', 'መ...  1.0  0.812834
4  ['የ', 'ዩ', 'ክ', 'ሬ', 'ን', '', 'ባ', 'ለ', 'ስ', '...  1.0  0.805243

   Substitutions  Insertions  Deletions
0              9           0         40
1             10           0         37
2             11           0         48
3              9           0         30
4              9           0         46

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 50 times
  ''ር']' -> 'ነበር': 23 times
  ''ው']' -> 'ናቸው': 10 times
  ''',' -> 'ጋር': 8 times
  ''',' -> 'ውስጥ': 7 times
  ''ል']' -> 'ብለዋል': 7 times
  ''ም']' -> 'አይደለም': 7 times
  ''',' -> 'ላይ': 6 times
  ''',' -> 'ወደ': 5 times
  ''ለ']' -> 'አለ': 5 times

Top 10 Preceding Contexts:
  Context ("'',", "'አ',"): 139 times
  Context ("'',", "'ነ',"): 118 times
  Context ("'ት',", "'',"): 94 times
  Context ("'ው',", "'',"): 83 times
  Context ("'ን',", "'',"): 76 times
  Context ("'',", "'ይ',"): 69 times
  Context ("'ም',", "'',"): 65 times
  Context ("'',", "'ተ',"): 63 times
  Context ("'ር',", "'',"): 45 times
  Context ("'ል',", "'',"): 36 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 51 times
  ''ር']' -> 'ነበር': 23 times
  ''ው']' -> 'ናቸ': 10 times
  ''',' -> 'ጋር': 8 times
  ''',' -> 'ውስጥ': 7 times
  ''ል']' -> 'ብለዋል': 7 times
  ''ም']' -> 'አይደለ': 7 times
  ''',' -> 'ላይ': 6 times
  ''',' -> 'ሺ': 6 times
  ''',' -> 'ወደ': 5 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 1373 times
  ''' -> '': 521 times
  ''' -> 'የ': 514 times
  ''' -> 'በ': 419 times
  ''' -> 'አ': 375 times
  '' -> 'ን': 302 times
  '' -> 'ው': 266 times
  '' -> 'ት': 253 times
  ''' -> 'እ': 251 times
  ''' -> 'መ': 251 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-5hrs-2_combined_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8812753342018733
Total Substitutions: 134584
Total Insertions: 0
Total Deletions: 673489
Total Words: 808073
Substitution Rate: 0.17
Insertion Rate: 0.00
Deletion Rate: 0.83

=== Sample Row-Level Results ===
                                          Prediction  \
0  ምግብ ከሱፐር ማርኬት መግዛት በተለምዱ እርካሹ ራስን የመመገበያ ዘዴ ነው...
1  ጃስሰቲናይንሲ ግሪፐን በመኮብፎበሜዳለይ በዘጠንተኩል ኤኤም አከባቢያዊ ሰአ...
2  ሰው አልባው የጨረቃሳ ተላይት ቻንድሪያን አንድ የሙን ኢንፓት ፒሮቡን ወይ...
3  ቅስፈታዊ የጹፍ አስተርጓሚ መተግበራዎች ሙሉ የጾህፍ ክፍሎችን ከአንድ ጓን...
4         ሶዌት ዩኔን ትልቅ ህብር የሶቬት ኔን ጀግና ለሰራው ስራሁሉ ተሰጠው

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.808000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.828784
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.816498
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.832168
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.842553

   Substitutions  Insertions  Deletions
0             22           0         82
1             20           0         65
2             29           0         95
3             15           0         74
4             10           0         39

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 605 times
  ''ር']' -> 'ነበር': 260 times
  ''ው']' -> 'ናቸው': 151 times
  ''',' -> 'ላይ': 129 times
  ''',' -> 'ነው': 126 times
  ''',' -> 'ውስጥ': 122 times
  ''',' -> 'ወደ': 112 times
  ''o']' -> 'ነው': 103 times
  ''ሉ']' -> 'ይችላሉ': 83 times
  ''ል']' -> 'ይችላል': 74 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 2109 times
  Context ("'',", "'አ',"): 1968 times
  Context ("'ን',", "'',"): 1836 times
  Context ("'',", "'m',"): 1624 times
  Context ("'',", "'ነ',"): 1581 times
  Context ("'o',", "'',"): 1438 times
  Context ("'',", "'የ',"): 1423 times
  Context ("'m',", "'a',"): 1412 times
  Context ("'',", "'ይ',"): 1288 times
  Context ("'e',", "'',"): 1272 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 612 times
  ''ር']' -> 'ነበር': 263 times
  ''',' -> 'ነ': 154 times
  ''ው']' -> 'ናቸ': 153 times
  ''',' -> 'ላይ': 130 times
  ''',' -> 'ውስጥ': 124 times
  ''',' -> 'ወደ': 114 times
  ''o']' -> 'ነ': 104 times
  ''',' -> 'እና': 88 times
  ''ሉ']' -> 'ይችላ': 83 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 46393 times
  ''' -> '': 20155 times
  ''' -> 'የ': 12528 times
  ''' -> 'በ': 10858 times
  '' -> 'ን': 10634 times
  ''' -> 'አ': 9639 times
  ''' -> 'ማ': 9198 times
  '' -> 'ው': 8238 times
  ''' -> 'ን': 8022 times
  ''' -> 'እ': 7784 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-200hrs-1_combined_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8714047356859348
Total Substitutions: 147422
Total Insertions: 0
Total Deletions: 660651
Total Words: 808073
Substitution Rate: 0.18
Insertion Rate: 0.00
Deletion Rate: 0.82

=== Sample Row-Level Results ===
                                          Prediction  \
0  ምግብ ከሶፐር ማርኬት መግዛት በተለምዶ ርከሹ እራስን የመመገቢያ ዘዴ ነው...
1  ጅስ ሳናይንሲ ግሪፐን የሞኮብኮበሜዳ ላይ በ ተኩል ኤኤም አከባቢያዊ ሰአት...
2  ሰው አልባው የጨረቃሳ ተላይት ቻንድሪርያን አንድ የሞ ኢምፓክት ፕሮቡን ኤ...
3  ቅፅበታዊ የፅሁፍ አስተርጓሚ መተግበሪያዎች ሙሉ የፅሁፍ ክፍሎችን ከአንድ ...
4  የሶቪየት ዩኒየን ትልቅ ክብር የሶቪየት ዩኒየን ጀግና ለሰራው ስራ ሁሉ ተሰጠው

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.802000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.818859
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.808081
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.792541
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.791489

   Substitutions  Insertions  Deletions
0             22           0         82
1             23           0         62
2             28           0         96
3             17           0         72
4             11           0         38

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 627 times
  ''',' -> 'እና': 270 times
  ''ር']' -> 'ነበር': 268 times
  ''',' -> 'ላይ': 202 times
  ''ው']' -> 'ናቸው': 158 times
  ''',' -> 'ነው': 151 times
  ''',' -> 'ውስጥ': 146 times
  ''',' -> 'ወደ': 126 times
  ''ሉ']' -> 'ይችላሉ': 105 times
  ''',' -> 'በ': 100 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 2216 times
  Context ("'',", "'አ',"): 2044 times
  Context ("'',", "'m',"): 1929 times
  Context ("'ን',", "'',"): 1906 times
  Context ("'o',", "'',"): 1791 times
  Context ("'m',", "'a',"): 1699 times
  Context ("'',", "'ነ',"): 1609 times
  Context ("'e',", "'',"): 1555 times
  Context ("'',", "'የ',"): 1514 times
  Context ("'',", "'ይ',"): 1313 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 629 times
  ''',' -> 'እና': 301 times
  ''ር']' -> 'ነበር': 269 times
  ''',' -> 'ላይ': 204 times
  ''',' -> 'ነ': 185 times
  ''ው']' -> 'ናቸ': 159 times
  ''',' -> 'ውስጥ': 146 times
  ''',' -> 'ወደ': 127 times
  ''',' -> 'በ': 113 times
  ''',' -> 'ን': 106 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 62923 times
  ''' -> '': 31769 times
  ''' -> 'የ': 15503 times
  ''' -> 'በ': 13106 times
  ''' -> 'አ': 11161 times
  ''' -> 'ን': 10233 times
  '' -> 'ን': 8620 times
  ''' -> 'ማ': 8455 times
  ''' -> 'እ': 8334 times
  ''' -> 'መ': 7352 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-5hrs-2_COMMON_VOICE_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8209692099771074
Total Substitutions: 4278
Total Insertions: 0
Total Deletions: 18302
Total Words: 22580
Substitution Rate: 0.19
Insertion Rate: 0.00
Deletion Rate: 0.81

=== Sample Row-Level Results ===
                                          Prediction  \
0   [UNK]ንለጉን እንዴጤንኛ ሆኖ ረዥማ መት መኖር እንጅላለን የሚለውም ይነሳል
1      ከተፈጠረበት ጊዜጀምሮ ለአራት ቢሊየን መታት ዮኖሩ የጠፈራ አካላት ናቸው
2  ሄሄሚቲ በመጨረሻም የጃንጃዊት ተከታይ ተብሎ ሊገለጽ የሚችሎ የፈጥሮ ደራሻ...
3           ሌሎች ደግሞ ከመሪነዋ ካርቶም እየሸሹ ወደ ሌሎ ችግዛቶቼ ይዳዋል
4  የዩኮሬን ባለስልጣናት መሪውና ተከታይያቻቸውን ከገዳ ሙ ለማስወጣትጥሬት ሲ...

                                           Reference  WER       CER  \
0  ['ጎ', 'ን', '', 'ለ', 'ጎ', 'ን', '', 'እ', 'ን', 'ዴ...  1.0  0.834043
1  ['ከ', 'ተ', 'ፈ', 'ጠ', 'ረ', 'በ', 'ት', '', 'ጊ', '...  1.0  0.814159
2  ['ሄ', 'ም', 'ቲ', '', 'በ', 'መ', 'ጨ', 'ረ', 'ሻ', '...  1.0  0.820423
3  ['ሌ', 'ሎ', 'ች', '', 'ደ', 'ግ', 'ሞ', '', 'ከ', 'መ...  1.0  0.823529
4  ['የ', 'ዩ', 'ክ', 'ሬ', 'ን', '', 'ባ', 'ለ', 'ስ', '...  1.0  0.805243

   Substitutions  Insertions  Deletions
0              9           0         40
1              9           0         38
2             12           0         47
3              9           0         30
4              9           0         46

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 51 times
  ''ር']' -> 'ነበር': 23 times
  ''ው']' -> 'ናቸው': 8 times
  ''ም']' -> 'አይደለም': 8 times
  ''ል']' -> 'ብለዋል': 7 times
  ''',' -> 'እና': 7 times
  ''',' -> 'ወደ': 5 times
  ''ሉ']' -> 'ይላሉ': 4 times
  ''ሩ']' -> 'ነበሩ': 4 times
  ''',' -> 'ደግሞ': 4 times

Top 10 Preceding Contexts:
  Context ("'',", "'አ',"): 136 times
  Context ("'',", "'ነ',"): 119 times
  Context ("'ት',", "'',"): 88 times
  Context ("'ው',", "'',"): 79 times
  Context ("'ን',", "'',"): 74 times
  Context ("'',", "'ይ',"): 70 times
  Context ("'ም',", "'',"): 64 times
  Context ("'',", "'ተ',"): 60 times
  Context ("'ር',", "'',"): 45 times
  Context ("'ነ',", "'በ',"): 35 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 51 times
  ''ር']' -> 'ነበር': 23 times
  ''ው']' -> 'ናቸ': 8 times
  ''ም']' -> 'አይደለ': 8 times
  ''ል']' -> 'ብለዋል': 7 times
  ''',' -> 'እና': 7 times
  ''',' -> 'ወደ': 5 times
  ''',' -> 'ደግሞ': 5 times
  ''ሉ']' -> 'ይላ': 4 times
  ''ሩ']' -> 'ነበሩ': 4 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 1329 times
  ''' -> '': 538 times
  ''' -> 'የ': 477 times
  ''' -> 'በ': 409 times
  ''' -> 'አ': 355 times
  '' -> 'ን': 310 times
  '' -> 'ው': 288 times
  ''' -> 'እ': 263 times
  ''' -> 'መ': 249 times
  '' -> 'ት': 248 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-5hrs-2_AMHARIC_DATASET_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8242154776819802
Total Substitutions: 39880
Total Insertions: 0
Total Deletions: 184413
Total Words: 224293
Substitution Rate: 0.18
Insertion Rate: 0.00
Deletion Rate: 0.82

=== Sample Row-Level Results ===
                                          Prediction  \
0  የኢንተርኔት አገልግሎ ተንም በተ መለከተ በክልሎች በዞኖችና በአዲስ አበባ...
1  በዚህም አነስተኛ ምስኖችን በማሷፋት አገሪቱን በተደጋጋሚ ከሚያጠቃት ድርቅ...
2  ይህ እንዳይሆን በህወታቸው መስዋእትነት ያስገኙትን የኢትዮጵያዊነት ክብር ...
3  የመጀመሪያው ነጥብ በተቃዋሚ ሀይሎች ሰፈር በጋራ አቋም ላይ  በጋራ ለማቋ...
4  አቅም በሚፈቅረው መሰረት አስቀድሞ መዘጋጀት ስለሚ ይፈልጉ የሚያጠራ አጥር...

                                           Reference  WER       CER  \
0  ['የ', 'ኢ', 'ን', 'ተ', 'ር', 'ኔ', 'ት', '', 'አ', '...  1.0  0.791762
1  ['በ', 'ዚ', 'ህ', 'ም', '', 'አ', 'ነ', 'ስ', 'ተ', '...  1.0  0.806045
2  ['ይ', 'ህ', '', 'እ', 'ን', 'ዳ', 'ይ', 'ሆ', 'ን', '...  1.0  0.802062
3  ['የ', 'መ', 'ጀ', 'መ', 'ሪ', 'ያ', 'ው', '', 'ነ', '...  1.0  0.791411
4  ['አ', 'ቅ', 'ም', '', 'በ', 'ሚ', 'ፈ', 'ቅ', 'ደ', '...  1.0  0.792818

   Substitutions  Insertions  Deletions
0             16           0         74
1             14           0         68
2             16           0         84
3             15           0         53
4             16           0         59

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 148 times
  ''ር']' -> 'ነበር': 51 times
  ''',' -> 'ነው': 48 times
  ''',' -> 'ላይ': 45 times
  ''ው']' -> 'ናቸው': 41 times
  ''',' -> 'ወደ': 40 times
  ''ዉ']' -> 'ነው': 37 times
  ''',' -> 'ውስጥ': 35 times
  ''',' -> 'ደግሞ': 27 times
  ''',' -> 'መንግስት': 25 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 943 times
  Context ("'',", "'አ',"): 888 times
  Context ("'ን',", "'',"): 771 times
  Context ("'',", "'የ',"): 695 times
  Context ("'',", "'ነ',"): 590 times
  Context ("'ው',", "'',"): 543 times
  Context ("'',", "'እ',"): 537 times
  Context ("'ም',", "'',"): 446 times
  Context ("'',", "'መ',"): 436 times
  Context ("'',", "'ይ',"): 427 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 154 times
  ''',' -> 'ነ': 55 times
  ''ር']' -> 'ነበር': 54 times
  ''',' -> 'ላይ': 46 times
  ''ው']' -> 'ናቸ': 43 times
  ''ዉ']' -> 'ነ': 41 times
  ''',' -> 'ወደ': 40 times
  ''',' -> 'ውስጥ': 35 times
  ''',' -> 'ደግሞ': 27 times
  ''',' -> 'መንግስት': 26 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 11736 times
  ''' -> 'የ': 4586 times
  ''' -> '': 4480 times
  ''' -> 'በ': 3945 times
  '' -> 'ን': 3375 times
  ''' -> 'አ': 3272 times
  '' -> 'ት': 3050 times
  '' -> 'ው': 2985 times
  ''' -> 'መ': 2768 times
  ''' -> 'እ': 2429 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-50hrs-3_FLEURS_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.9733414743035992
Total Substitutions: 41854
Total Insertions: 0
Total Deletions: 265356
Total Words: 307210
Substitution Rate: 0.14
Insertion Rate: 0.00
Deletion Rate: 0.86

=== Sample Row-Level Results ===
                                          Prediction  \
0  ድማ ማ መ አ እር  ን  ራአለ ያር ብረ እድን ደ አአ ራራር ባር ራ ኢአ...
1  ኦሰቺውር ፓሰሚ ብልኒ የመኮ አልፈ ከከደሚያይ እነብር እወን ኦገ  ኬደ ኮ...
2     ፒቸማቺ የ የውከአራፎን ኬንበ ዋዲ ሪፎርድንስ እንደሞለው የከካ ቢቻው ጉበ
3  ሜዴላ ሞኮማ በነደ ሲና ሄኛ የፔን ኢውው አዶ ላማር አሜረጋ ለቪን ሚሊዮን...
4  ሲብች  ላ ጆቤ ነን ማቸም መየግሄ አሄኛ ነትእኛው አእ የአላው አኛላ ን ...

                                           Reference  WER       CER  \
0  ['m', 'a', 'd', 'h', '', 'm', 'a', 't', 'h', '...  1.0  0.967236
1  ['o', 's', 'e', 'c', 'h', 'i', 'w', '', 'r', '...  1.0  0.968439
2  ['p', 'i', 'c', 'h', 'a', '', 'm', 'a', 'c', '...  1.0  0.980044
3  ['m', 'i', 'd', 'i', 'l', 'a', '', 'm', 'o', '...  1.0  0.968310
4  ['s', 'i', 'b', 'u', 'o', 'c', 'h', 'e', '', '...  1.0  0.971557

   Substitutions  Insertions  Deletions
0             21           0        126
1             18           0        107
2             10           0         83
3             19           0         99
4             18           0        120

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''',' -> 'ር': 99 times
  ''',' -> 'እ': 95 times
  ''',' -> 'ነ': 91 times
  ''',' -> 'አ': 71 times
  ''o',' -> 'ር': 70 times
  ''o',' -> 'እ': 58 times
  ''',' -> 'ን': 52 times
  ''o',' -> 'ነ': 52 times
  ''a',' -> 'ር': 50 times
  ''a',' -> 'እ': 49 times

Top 10 Preceding Contexts:
  Context ("'',", "'m',"): 1642 times
  Context ("'o',", "'',"): 1464 times
  Context ("'m',", "'a',"): 1440 times
  Context ("'e',", "'',"): 1284 times
  Context ("'a',", "'',"): 841 times
  Context ("'',", "'k',"): 823 times
  Context ("'i',", "'',"): 723 times
  Context ("'a',", "'r',"): 626 times
  Context ("'r',", "'',"): 561 times
  Context ("'c',", "'h',"): 548 times

Top 10 Substitution Root Patterns:
  ''',' -> 'እ': 137 times
  ''',' -> 'ነ': 124 times
  ''',' -> 'ር': 104 times
  ''',' -> 'አ': 81 times
  ''o',' -> 'ር': 75 times
  ''o',' -> 'እ': 72 times
  ''o']' -> 'ነ': 68 times
  ''o',' -> 'ነ': 64 times
  ''i',' -> 'እ': 60 times
  ''a',' -> 'ነ': 60 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 27516 times
  ''' -> '': 18867 times
  ''' -> 'እ': 4509 times
  ''' -> 'አ': 4392 times
  ''' -> 'የ': 3090 times
  ''' -> 'ማ': 3017 times
  ''' -> 'ር': 2621 times
  ''' -> 'ነ': 2253 times
  ''' -> 'ን': 2205 times
  ''' -> 'ከ': 1696 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-10hrs-3_FLEURS_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.9722554112349442
Total Substitutions: 43457
Total Insertions: 0
Total Deletions: 263753
Total Words: 307210
Substitution Rate: 0.14
Insertion Rate: 0.00
Deletion Rate: 0.86

=== Sample Row-Level Results ===
                                          Prediction  \
0  አዝማት ማግ ከሆሙ ማዕቅ ዖካሮው ዶም ከሞ ትዮሙ ፖ ጎሮ ኤይዴንድ ናችከኛ...
1  ኦሰች ውሪፓድኒ ደልን የሟኮ አልፎቼ ከ[UNK]ድሚያንወን መዲሬ ኤይጎን ነ...
2  ቢቸማች የለሆኚ ስከአቸውም ስ ኮሎፎቶን ጌንድ ወዲ ሪፎተውንስዴ ንደሞላዊ ...
3  ሚዲላም ሞኮማ በንደ ቲና ይሄኛ ኤፒን ኢውስ ኤዶላማር አሜርጋ ሌቬን ሚሊዮ...
4  ሲቦዎች የማሞንፊሌ ጀቤድ ጋ አቻልረ መየግን አሄኝ አቶ ጌኛው አልጌ ማላ ...

                                           Reference  WER       CER  \
0  ['m', 'a', 'd', 'h', '', 'm', 'a', 't', 'h', '...  1.0  0.971510
1  ['o', 's', 'e', 'c', 'h', 'i', 'w', '', 'r', '...  1.0  0.966777
2  ['p', 'i', 'c', 'h', 'a', '', 'm', 'a', 'c', '...  1.0  0.975610
3  ['m', 'i', 'd', 'i', 'l', 'a', '', 'm', 'o', '...  1.0  0.959507
4  ['s', 'i', 'b', 'u', 'o', 'c', 'h', 'e', '', '...  1.0  0.971557

   Substitutions  Insertions  Deletions
0             21           0        126
1             19           0        106
2             12           0         81
3             22           0         96
4             20           0        118

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''o']' -> 'ነው': 117 times
  ''',' -> 'ማር': 57 times
  ''',' -> 'ነው': 54 times
  ''',' -> 'ነ': 52 times
  ''o',' -> 'ነ': 44 times
  ''o',' -> 'ማር': 37 times
  ''a',' -> 'ማር': 35 times
  ''e',' -> 'ነው': 31 times
  ''i',' -> 'ማር': 30 times
  ''i',' -> 'ነው': 26 times

Top 10 Preceding Contexts:
  Context ("'',", "'m',"): 1712 times
  Context ("'o',", "'',"): 1541 times
  Context ("'m',", "'a',"): 1511 times
  Context ("'e',", "'',"): 1364 times
  Context ("'a',", "'',"): 862 times
  Context ("'',", "'k',"): 857 times
  Context ("'i',", "'',"): 762 times
  Context ("'a',", "'r',"): 648 times
  Context ("'c',", "'h',"): 572 times
  Context ("'',", "'n',"): 570 times

Top 10 Substitution Root Patterns:
  ''o']' -> 'ነ': 118 times
  ''',' -> 'ነ': 110 times
  ''o',' -> 'ነ': 70 times
  ''',' -> 'ማር': 57 times
  ''e',' -> 'ነ': 53 times
  ''a',' -> 'ነ': 50 times
  ''i',' -> 'ነ': 46 times
  ''o',' -> 'ማር': 40 times
  ''a',' -> 'ማር': 35 times
  ''n',' -> 'ነ': 34 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 20998 times
  ''' -> '': 11245 times
  ''' -> 'ማ': 5559 times
  ''' -> 'ን': 3657 times
  ''' -> 'የ': 3197 times
  '' -> 'ን': 2651 times
  ''' -> 'ወ': 2468 times
  ''' -> 'ከ': 2381 times
  ''' -> 'አ': 2220 times
  ',' -> 'ን': 1995 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-5hrs-2_FLEURS_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.97381764001467
Total Substitutions: 41080
Total Insertions: 0
Total Deletions: 266130
Total Words: 307210
Substitution Rate: 0.13
Insertion Rate: 0.00
Deletion Rate: 0.87

=== Sample Row-Level Results ===
                                          Prediction  \
0  መድማት ማግከሆኖማኦክኦካሎ  ዶንኮኖቲዬሙ ፖ[UNK]ሬ እ ዴንድ ናቸውከንና...
1  ኦሰቺውሪፓድኒ ዴልኒ ማኮ አልፎቺ ከኮድሚያን መንዲዬሬ ኢ[UNK]ንናው ኦን...
2  ቤቻማች የሎኝ ሰቀከ አቶውምስ ኮሎፎቶንስ ጌንደ ዋዲ ሪፎተወንስዲ እንደው ...
3  ሜዴላ አሞኮማ በንደቴና ሂኛ የፔን ው ሰው ኤዶላማር አመሪጋ ሌቬን ሚሊዮን...
4  ሲቦዎቼማሞን ፊ ሌ ዘቤዶጋ ማቻልድሪ መሲዬግኒ አሄኛ ናተው ጌኛው አልጌ ማ...

                                           Reference  WER       CER  \
0  ['m', 'a', 'd', 'h', '', 'm', 'a', 't', 'h', '...  1.0  0.972934
1  ['o', 's', 'e', 'c', 'h', 'i', 'w', '', 'r', '...  1.0  0.970100
2  ['p', 'i', 'c', 'h', 'a', '', 'm', 'a', 'c', '...  1.0  0.973392
3  ['m', 'i', 'd', 'i', 'l', 'a', '', 'm', 'o', '...  1.0  0.964789
4  ['s', 'i', 'b', 'u', 'o', 'c', 'h', 'e', '', '...  1.0  0.974551

   Substitutions  Insertions  Deletions
0             15           0        132
1             17           0        108
2             11           0         82
3             19           0         99
4             18           0        120

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''o']' -> 'ነው': 103 times
  ''',' -> 'ማር': 49 times
  ''',' -> 'ነው': 45 times
  ''o',' -> 'ማር': 41 times
  ''',' -> 'ማ': 35 times
  ''a',' -> 'ነው': 34 times
  ''o',' -> 'ነው': 32 times
  ''a',' -> 'ማር': 30 times
  ''o',' -> 'ማ': 25 times
  ''e',' -> 'ነው': 25 times

Top 10 Preceding Contexts:
  Context ("'',", "'m',"): 1624 times
  Context ("'o',", "'',"): 1438 times
  Context ("'m',", "'a',"): 1412 times
  Context ("'e',", "'',"): 1272 times
  Context ("'a',", "'',"): 815 times
  Context ("'',", "'k',"): 788 times
  Context ("'i',", "'',"): 709 times
  Context ("'a',", "'r',"): 624 times
  Context ("'c',", "'h',"): 546 times
  Context ("'r',", "'',"): 539 times

Top 10 Substitution Root Patterns:
  ''o']' -> 'ነ': 104 times
  ''',' -> 'ነ': 66 times
  ''',' -> 'ማር': 54 times
  ''a',' -> 'ነ': 47 times
  ''o',' -> 'ማር': 44 times
  ''o',' -> 'ነ': 42 times
  ''',' -> 'ማ': 39 times
  ''a',' -> 'ማር': 32 times
  ''e',' -> 'ነ': 31 times
  ''o',' -> 'ማ': 30 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 17639 times
  ''' -> '': 9175 times
  ''' -> 'ማ': 6263 times
  ''' -> 'ን': 3304 times
  '' -> 'ን': 3203 times
  '' -> 'ው': 2334 times
  ''' -> 'እ': 2327 times
  ''' -> 'የ': 2287 times
  ''' -> 'አ': 2068 times
  ''' -> 'ወ': 1848 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-100hrs_COMMON_VOICE_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8115731504380844
Total Substitutions: 4513
Total Insertions: 0
Total Deletions: 18067
Total Words: 22580
Substitution Rate: 0.20
Insertion Rate: 0.00
Deletion Rate: 0.80

=== Sample Row-Level Results ===
                                          Prediction  \
0    ጎንለቡን እንዴት ጤነኛ ሆነ ረዥም ዓመት መኖር እንችላለን የሚለውም ይነሣል
1     ከተፈጠረበት ጊዜ ጀምሮ ለአራት ቢሊየን መታት የኖሩ የጠፈር አካላት ናቸው
2  ሄሚቲ በመጨረሻም የጃንጃዊት ተከታይ ተብሎ ሊገለፅ የሚችሉ የፈጥሮ ደራሻ ...
3           ሌሎች ደግሞ ከመዲንዋ ካርቶም እየሸሹ ወደ ሌሎች ግዛቶች ሄደዋል
4  የዩክሬን ባለስልጣናት መሪውና ተከታይ ርቻቸውን ከገዳሞ ለማስወጣት ጥሬት ...

                                           Reference  WER       CER  \
0  ['ጎ', 'ን', '', 'ለ', 'ጎ', 'ን', '', 'እ', 'ን', 'ዴ...  1.0  0.812766
1  ['ከ', 'ተ', 'ፈ', 'ጠ', 'ረ', 'በ', 'ት', '', 'ጊ', '...  1.0  0.800885
2  ['ሄ', 'ም', 'ቲ', '', 'በ', 'መ', 'ጨ', 'ረ', 'ሻ', '...  1.0  0.813380
3  ['ሌ', 'ሎ', 'ች', '', 'ደ', 'ግ', 'ሞ', '', 'ከ', 'መ...  1.0  0.807487
4  ['የ', 'ዩ', 'ክ', 'ሬ', 'ን', '', 'ባ', 'ለ', 'ስ', '...  1.0  0.801498

   Substitutions  Insertions  Deletions
0             10           0         39
1             10           0         37
2             12           0         47
3              9           0         30
4             10           0         45

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 52 times
  ''ር']' -> 'ነበር': 21 times
  ''ው']' -> 'ናቸው': 10 times
  ''ም']' -> 'አይደለም': 9 times
  ''',' -> 'ጋር': 8 times
  ''',' -> 'እና': 7 times
  ''',' -> 'ወደ': 6 times
  ''ል']' -> 'ብለዋል': 6 times
  ''ሉ']' -> 'ያስረዳሉ': 6 times
  ''ለ']' -> 'አለ': 5 times

Top 10 Preceding Contexts:
  Context ("'',", "'አ',"): 139 times
  Context ("'',", "'ነ',"): 120 times
  Context ("'ት',", "'',"): 96 times
  Context ("'ው',", "'',"): 83 times
  Context ("'ን',", "'',"): 82 times
  Context ("'ም',", "'',"): 70 times
  Context ("'',", "'ይ',"): 69 times
  Context ("'',", "'ተ',"): 62 times
  Context ("'ር',", "'',"): 46 times
  Context ("'ል',", "'',"): 38 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 55 times
  ''ር']' -> 'ነበር': 21 times
  ''ው']' -> 'ናቸ': 10 times
  ''ም']' -> 'አይደለ': 9 times
  ''',' -> 'ጋር': 8 times
  ''',' -> 'እና': 7 times
  ''',' -> 'ወደ': 6 times
  ''ል']' -> 'ብለዋል': 6 times
  ''ሉ']' -> 'ያስረዳ': 6 times
  ''',' -> 'ሁለት': 6 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 1506 times
  ''' -> '': 603 times
  ''' -> 'የ': 517 times
  ''' -> 'በ': 413 times
  ''' -> 'አ': 404 times
  ''' -> 'እ': 297 times
  '' -> 'ው': 277 times
  ''' -> 'መ': 260 times
  '' -> 'ን': 256 times
  '' -> 'ት': 249 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-2_AMHARIC_ALFA_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8420352052876782
Total Substitutions: 44322
Total Insertions: 0
Total Deletions: 209668
Total Words: 253990
Substitution Rate: 0.17
Insertion Rate: 0.00
Deletion Rate: 0.83

=== Sample Row-Level Results ===
                                          Prediction  \
0  በእግብ ከስሱፕርማርኪት መግዘት በተለምዱ እርካሹራስን የመመገብ ያዘደ ምግ...
1  ሸያህሱ  ሳቲናዩንሴ ግሪፕን በማኮብኮጋ ሜዳለ  በዘጠን ተኩል ኤየኤ አከባ...
2  በሰሰ አልባው የጨረቃሳ ትላይትቻንድሪያን አንየሙን ኢንፓርትፒሮቡን ይም ኤ...
3  ቅስፈታዊ የጽፋስተርጓሚ መተግበሪዎዎች ሙሩ የጽፍ ክፍሎችን ከዱ ንቃ ዋሊዳ...
4      ሶቤት ውኒ የን ትልቅ ህብር የሶቤት ዩኒን ጀግና ለስራው ስራሁን ተሰጠው

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.838000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.826303
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.850168
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.850816
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.834043

   Substitutions  Insertions  Deletions
0             16           0         88
1             22           0         63
2             23           0        101
3             14           0         75
4             11           0         38

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 290 times
  ''ር']' -> 'ነበር': 76 times
  ''ው']' -> 'ናቸው': 63 times
  ''ል']' -> 'ይችላል': 56 times
  ''ር']' -> 'ነበል': 38 times
  ''ሉ']' -> 'ይችላሉ': 38 times
  ''',' -> 'እና': 30 times
  ''',' -> 'ውስጥ': 30 times
  ''',' -> 'ጊዜ': 29 times
  ''',' -> 'ብዙ': 29 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 965 times
  Context ("'ን',", "'',"): 892 times
  Context ("'',", "'አ',"): 879 times
  Context ("'',", "'ነ',"): 854 times
  Context ("'',", "'ይ',"): 776 times
  Context ("'',", "'የ',"): 561 times
  Context ("'',", "'ተ',"): 523 times
  Context ("'',", "'እ',"): 500 times
  Context ("'ው',", "'',"): 491 times
  Context ("'ች',", "'',"): 479 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 292 times
  ''ር']' -> 'ነበር': 82 times
  ''ው']' -> 'ናቸ': 68 times
  ''ል']' -> 'ይችላል': 56 times
  ''ሉ']' -> 'ይችላ': 46 times
  ''ር']' -> 'ነበል': 38 times
  ''',' -> 'እና': 34 times
  ''',' -> 'ብዙ': 32 times
  ''',' -> 'ጊዜ': 31 times
  ''',' -> 'ውስጥ': 30 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 12512 times
  ''' -> 'የ': 5413 times
  ''' -> 'በ': 5203 times
  ''' -> '': 5173 times
  '' -> 'ን': 4768 times
  '' -> 'ት': 3494 times
  ''' -> 'አ': 3307 times
  ''' -> 'መ': 2994 times
  '' -> 'ው': 2798 times
  '' -> 'ች': 2567 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-200hrs-1_AMHARIC_DATASET_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8220229666916251
Total Substitutions: 41710
Total Insertions: 0
Total Deletions: 182583
Total Words: 224293
Substitution Rate: 0.19
Insertion Rate: 0.00
Deletion Rate: 0.81

=== Sample Row-Level Results ===
                                          Prediction  \
0  የኢንተርኔት አገልግሎትንም በተመለከተ በክልሎች በዞኖችና በአዲስ አበባ በ...
1  በዚህም አነስተኛ ምስኖችን በማስዋፋት አገሪቱን በተደጋጋሚ ከሚያጠቃት ድር...
2  ይህ እንዳይሆን በህይወታቸው መስዋእትነት ያስገኙትን የኢትዮጵያዊነት ክብር...
3  የመጀመሪያው ነጥብ በተቃዋሚ ሀይሎች ሰፈር በጋራ አቋም ላይ በጋራ ለማቆም...
4  አቅን በሚፈቅደው መሰረት አስቀድሞው መዘጋጀትን ስለሚ ፈልጉ የሚያጠራጥር ...

                                           Reference  WER       CER  \
0  ['የ', 'ኢ', 'ን', 'ተ', 'ር', 'ኔ', 'ት', '', 'አ', '...  1.0  0.794050
1  ['በ', 'ዚ', 'ህ', 'ም', '', 'አ', 'ነ', 'ስ', 'ተ', '...  1.0  0.801008
2  ['ይ', 'ህ', '', 'እ', 'ን', 'ዳ', 'ይ', 'ሆ', 'ን', '...  1.0  0.804124
3  ['የ', 'መ', 'ጀ', 'መ', 'ሪ', 'ያ', 'ው', '', 'ነ', '...  1.0  0.791411
4  ['አ', 'ቅ', 'ም', '', 'በ', 'ሚ', 'ፈ', 'ቅ', 'ደ', '...  1.0  0.795580

   Substitutions  Insertions  Deletions
0             14           0         76
1             14           0         68
2             15           0         85
3             15           0         53
4             15           0         60

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 133 times
  ''',' -> 'ላይ': 58 times
  ''',' -> 'ነው': 57 times
  ''ር']' -> 'ነበር': 46 times
  ''',' -> 'ወደ': 45 times
  ''ው']' -> 'ናቸው': 41 times
  ''',' -> 'ግን': 36 times
  ''',' -> 'ውስጥ': 34 times
  ''ዉ']' -> 'ነው': 28 times
  ''',' -> 'ደግሞ': 26 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 992 times
  Context ("'',", "'አ',"): 940 times
  Context ("'ን',", "'',"): 809 times
  Context ("'',", "'የ',"): 738 times
  Context ("'',", "'ነ',"): 601 times
  Context ("'ው',", "'',"): 572 times
  Context ("'',", "'እ',"): 562 times
  Context ("'ም',", "'',"): 479 times
  Context ("'',", "'መ',"): 460 times
  Context ("'',", "'ይ',"): 438 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 134 times
  ''',' -> 'ነ': 63 times
  ''',' -> 'ላይ': 59 times
  ''ር']' -> 'ነበር': 47 times
  ''',' -> 'ወደ': 45 times
  ''ው']' -> 'ናቸ': 42 times
  ''',' -> 'ግን': 36 times
  ''',' -> 'ውስጥ': 34 times
  ''ዉ']' -> 'ነ': 29 times
  ''',' -> 'ደግሞ': 28 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 13202 times
  ''' -> 'የ': 5157 times
  ''' -> '': 5043 times
  ''' -> 'አ': 4204 times
  ''' -> 'በ': 4188 times
  '' -> 'ን': 3060 times
  '' -> 'ው': 2969 times
  '' -> 'ት': 2871 times
  ''' -> 'መ': 2805 times
  ''' -> 'እ': 2304 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-100hrs_combined_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8717732565465816
Total Substitutions: 140435
Total Insertions: 0
Total Deletions: 667638
Total Words: 808073
Substitution Rate: 0.17
Insertion Rate: 0.00
Deletion Rate: 0.83

=== Sample Row-Level Results ===
                                          Prediction  \
0  ምግብ ከሱፐርማርኬት መግዛት በተለምዶ ርካሹ ራስን የመመገበ ዘዴ ነው ምግ...
1  ጃስሰይ ሪፐን በማኮብፖበ ሜዳ ላይ በኝ ተኩል ኤኤም አከባቢያዊ ሰአት በዜ...
2  ሰው አልባው የጨረቃ ተላይትቻንድሪያን አንድ የሞ ኢንፓክት ፒሮቡን ወይም ...
3  ቅፅበታዊ የፅሁፍ አስተርጓሚ መተግበሪያዎች ሙሉ የፅሁፍ ክፍሎችን ከአንድ ...
4     ሶውዬት ዩኒን ትልቅ ክብር የሶቬት ዩኒየን ጀግና ለስራው ስራ ሁሉ ተሰጠው

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.804000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.808933
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.816498
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.794872
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.821277

   Substitutions  Insertions  Deletions
0             21           0         83
1             23           0         62
2             28           0         96
3             17           0         72
4             11           0         38

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 637 times
  ''',' -> 'እና': 300 times
  ''ር']' -> 'ነበር': 267 times
  ''',' -> 'ላይ': 226 times
  ''',' -> 'ነው': 192 times
  ''ው']' -> 'ናቸው': 158 times
  ''',' -> 'ወደ': 128 times
  ''',' -> 'ውስጥ': 121 times
  ''o']' -> 'ነው': 107 times
  ''ሉ']' -> 'ይችላሉ': 102 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 2323 times
  Context ("'',", "'አ',"): 2097 times
  Context ("'ን',", "'',"): 1991 times
  Context ("'',", "'ነ',"): 1621 times
  Context ("'',", "'የ',"): 1608 times
  Context ("'',", "'m',"): 1531 times
  Context ("'m',", "'a',"): 1362 times
  Context ("'o',", "'',"): 1343 times
  Context ("'',", "'ይ',"): 1324 times
  Context ("'',", "'እ',"): 1296 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 644 times
  ''',' -> 'እና': 327 times
  ''ር']' -> 'ነበር': 270 times
  ''',' -> 'ላይ': 227 times
  ''',' -> 'ነ': 226 times
  ''ው']' -> 'ናቸ': 159 times
  ''',' -> 'ወደ': 129 times
  ''',' -> 'ውስጥ': 121 times
  ''o']' -> 'ነ': 110 times
  ''',' -> 'ጊዜ': 107 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 56211 times
  ''' -> '': 26219 times
  ''' -> 'የ': 14481 times
  ''' -> 'አ': 11151 times
  ''' -> 'በ': 10796 times
  ''' -> 'መ': 8305 times
  ''' -> 'ን': 8216 times
  ''' -> 'እ': 8170 times
  '' -> 'ን': 8001 times
  ''' -> 'ማ': 7114 times

====================================
Analyzing results from: csv_results/wav2vec2-xls-r-300m-amharic-asr-dataset-10hrs-3_combined_results.csv

=== Overall Metrics ===
Overall WER: 1.0
Overall CER: 0.8755111844666785
Total Substitutions: 142566
Total Insertions: 0
Total Deletions: 665507
Total Words: 808073
Substitution Rate: 0.18
Insertion Rate: 0.00
Deletion Rate: 0.82

=== Sample Row-Level Results ===
                                          Prediction  \
0  ምግብ ከሱፐር ማርኬት መግዛት በተለምዶ ርካሹ እራስን የመመገብያዘሬ ነው ...
1  ጃስ ሰቲናይንሲ ግሪፐን በማኮብፎ በያ ሜዳ ለይ በዘጠን ተኩል የኤም አከባ...
2  ሰውአልባው የጨረቃሳ ትላይት ቻንድሪያን አንድ የሞን ኢምፓት ፒሮቡን ወይም...
3  ቅስፈታዊ የፅሁፍ አስተርጓሚ መተግብራዎች ሙሩ የፅሁፍ ክፍሎችን ከአድጓንቀ...
4       ሶቪየት ዩኔን ትልቅክብር  የሶቪየት ዩኔን ጀግና ለሰራው ስራሁሉ ተሰጠ

                                           Reference  WER       CER  \
0  ['ም', 'ግ', 'ብ', '', 'ከ', 'ሱ', 'ፐ', 'ር', 'ማ', '...  1.0  0.802000
1  ['', '', '', 'በ', 'ማ', 'ኮ', 'ብ', 'ኮ', 'ብ', 'ያ'...  1.0  0.811414
2  ['ሰ', 'ው', '', 'አ', 'ል', 'ባ', 'ው', '', 'የ', 'ጨ...  1.0  0.808081
3  ['ቅ', 'ፅ', 'በ', 'ታ', 'ዊ', '', 'የ', 'ፅ', 'ሁ', '...  1.0  0.827506
4  ['የ', 'ሶ', 'ቪ', 'የ', 'ት', '', 'ዩ', 'ኒ', 'የ', '...  1.0  0.821277

   Substitutions  Insertions  Deletions
0             22           0         82
1             26           0         59
2             31           0         93
3             14           0         75
4              9           0         40

=== Deeper Substitution Analysis ===

Top 10 Substitution Word Patterns:
  ''ው']' -> 'ነው': 638 times
  ''ር']' -> 'ነበር': 267 times
  ''',' -> 'ላይ': 198 times
  ''ው']' -> 'ናቸው': 160 times
  ''',' -> 'ውስጥ': 159 times
  ''',' -> 'ነው': 159 times
  ''',' -> 'ወደ': 129 times
  ''',' -> 'እና': 124 times
  ''o']' -> 'ነው': 117 times
  ''',' -> 'ወይም': 97 times

Top 10 Preceding Contexts:
  Context ("'ት',", "'',"): 2245 times
  Context ("'',", "'አ',"): 2053 times
  Context ("'ን',", "'',"): 1931 times
  Context ("'',", "'m',"): 1712 times
  Context ("'',", "'ነ',"): 1598 times
  Context ("'',", "'የ',"): 1566 times
  Context ("'o',", "'',"): 1541 times
  Context ("'m',", "'a',"): 1511 times
  Context ("'e',", "'',"): 1364 times
  Context ("'',", "'ይ',"): 1317 times

Top 10 Substitution Root Patterns:
  ''ው']' -> 'ነ': 643 times
  ''ር']' -> 'ነበር': 270 times
  ''',' -> 'ነ': 227 times
  ''',' -> 'ላይ': 199 times
  ''ው']' -> 'ናቸ': 162 times
  ''',' -> 'ውስጥ': 159 times
  ''',' -> 'እና': 150 times
  ''',' -> 'ወደ': 130 times
  ''o']' -> 'ነ': 118 times
  ''',' -> 'ወይ': 109 times

Top 10 Character Mismatch Pairs:
  ',' -> '': 53917 times
  ''' -> '': 23813 times
  ''' -> 'የ': 14993 times
  ''' -> 'በ': 11094 times
  ''' -> 'አ': 10453 times
  '' -> 'ን': 9383 times
  ''' -> 'ን': 8677 times
  ''' -> 'መ': 8459 times
  ''' -> 'ማ': 8311 times
  ''' -> 'እ': 7349 times
"""

# --- Split the raw text into blocks ---
# Each block begins with "Analyzing results from:" so we split by that marker.
blocks = re.split(r'Analyzing results from:', raw_text)[1:]

results = []

# Process each block
for block in blocks:
    block = block.strip()
    lines = block.splitlines()

    # The first non-empty line is assumed to be the file path.
    file_line = lines[0].strip()

    # Extract the model and dataset names from the filename.
    # We assume filenames follow the pattern: <model>_<dataset>_results.csv
    m = re.search(r'([^/]+)_([^_]+)_results\.csv', file_line)
    if m:
        model_name = m.group(1)
        dataset_name = m.group(2)
    else:
        model_name = "Unknown"
        dataset_name = "Unknown"

    # Extract the number of hours from the model name if available.
    # This looks for a pattern like "-<number>hrs" (e.g., "-1hrs", "-20hrs", etc.)
    hours_match = re.search(r'-(\d+)hrs', model_name)
    if hours_match:
        hours = int(hours_match.group(1))
    else:
        hours = None

    try:
        overall_cer = float(re.search(r'Overall CER:\s*([\d\.]+)', block).group(1))
        total_words = int(re.search(r'Total Words:\s*([\d,]+)', block).group(1).replace(',', ''))
        substitutions = int(re.search(r'Total Substitutions:\s*([\d,]+)', block).group(1).replace(',', ''))
        insertions = int(re.search(r'Total Insertions:\s*([\d,]+)', block).group(1).replace(',', ''))
        deletions = int(re.search(r'Total Deletions:\s*([\d,]+)', block).group(1).replace(',', ''))
        sub_rate = float(re.search(r'Substitution Rate:\s*([\d\.]+)', block).group(1))
        insertion_rate = float(re.search(r'Insertion Rate:\s*([\d\.]+)', block).group(1))
        del_rate = float(re.search(r'Deletion Rate:\s*([\d\.]+)', block).group(1))
    except Exception as e:
        print(f"Error extracting numbers from block for {model_name} - {dataset_name}: {e}")
        continue

    results.append({
        "Model": model_name,
        "Dataset": dataset_name,
        "Hours": hours,
        "Overall CER": overall_cer,
        "Total Words": total_words,
        "Substitutions": substitutions,
        "Insertions": insertions,
        "Deletions": deletions,
        "Sub Rate": sub_rate,
        "Insertion Rate": insertion_rate,
        "Del Rate": del_rate
    })

# Create the summary DataFrame
df_summary = pd.DataFrame(results)

# Sort the DataFrame by Hours first (grouping similar models together) and then by Model and Dataset.
df_summary = df_summary.sort_values(by=["Hours", "Model", "Dataset"])

# Display the summary table
print("\nSummary Table:")
print(df_summary)

# Optionally, save the summary table to a CSV file.
df_summary.to_csv("summary_table.csv", index=False)



Summary Table:
                                                Model   Dataset  Hours  \
19     wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-2    FLEURS      1   
3      wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-2  combined      1   
31  wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-2...      ALFA      1   
14  wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-2...   DATASET      1   
1   wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-2...     VOICE      1   
29     wav2vec2-xls-r-300m-amharic-asr-dataset-5hrs-2    FLEURS      5   
23     wav2vec2-xls-r-300m-amharic-asr-dataset-5hrs-2  combined      5   
20  wav2vec2-xls-r-300m-amharic-asr-dataset-5hrs-2...      ALFA      5   
26  wav2vec2-xls-r-300m-amharic-asr-dataset-5hrs-2...   DATASET      5   
25  wav2vec2-xls-r-300m-amharic-asr-dataset-5hrs-2...     VOICE      5   
28    wav2vec2-xls-r-300m-amharic-asr-dataset-10hrs-3    FLEURS     10   
34    wav2vec2-xls-r-300m-amharic-asr-dataset-10hrs-3  combined     10   
4   wav2vec2-xls-r-300

# **AMHARIC**

In [ ]:
from datasets import load_dataset, DatasetDict
from datasets import load_dataset, Audio

SAMPLE_RATE = 16000

# Load dataset in streaming mode
test5hr = load_dataset("KYAGABA/fluers_cleaned_testset_amharic", split="train", streaming=True)

# Cast audio column
test5hr = test5hr.cast_column("audio", Audio(sampling_rate=SAMPLE_RATE))

# Function to select 5 hours of test audio
def select_audio_by_hours(dataset, target_hours):
    target_duration = target_hours * 3600  # Convert hours to seconds
    cumulative_duration = 0
    selected_samples = []

    for example in iter(dataset):  # Streaming iteration
        audio_duration = len(example["audio"]["array"]) / SAMPLE_RATE  # Convert to seconds
        cumulative_duration += audio_duration

        selected_samples.append(example)
        if cumulative_duration >= target_duration:
            break

    return selected_samples

# Select exactly 5 hours of test audio
test5hr_samples = select_audio_by_hours(test5hr, target_hours=5)

# Print total duration
total_duration_test = sum(len(sample["audio"]["array"]) for sample in test5hr_samples) / SAMPLE_RATE / 3600
print(f"Total Test Set Duration: {total_duration_test:.2f} hours")

# Convert to Hugging Face Dataset format
from datasets import Dataset
test5hr = Dataset.from_list(test5hr_samples)
print(test5hr)


# test5hr = load_dataset("KYAGABA/fluers_cleaned_testset_amharic")['']
# print(test5hr)
# test5hr=test5hr.cast_column("audio", Audio(sampling_rate=16_000))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Total Test Set Duration: 5.00 hours


In [ ]:
SAMPLE_RATE=16000
total_duration_train = sum(len(example["audio"]["array"]) for example in test5hr) / SAMPLE_RATE / 3600
total_duration_train

NameError: name 'test5hr' is not defined

In [ ]:
import unicodedata
import re
def preprocess_text(batch):
    batch['transcription'] = [text.lower() for text in batch['transcription']]
    batch['transcription'] = [unicodedata.normalize('NFKC', text) for text in batch['transcription']]
    batch['transcription'] = [re.sub(r'[\’\ʻ\ʼ\ʽ\‘\']', '', text) for text in batch['transcription']]
    batch['transcription'] = [re.sub(r'[^\w\s\']', '', text) for text in batch['transcription']]
    batch['transcription'] = [re.sub(r'_', ' ', text) for text in batch['transcription']]
    batch['transcription'] = [' '.join(text.split()) for text in batch['transcription']]
    return batch

test5hr = test5hr.map(preprocess_text, batched=True)

In [ ]:
def to_lowercase(example):
    example["transcription"] = example["transcription"].lower()
    return example

test5hr = test5hr.map(to_lowercase)

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}
vocab_test = test5hr.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test5hr.column_names)
vocab_list = list(set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

In [ ]:
# models = [
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_1hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_5hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_10hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_20hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_NCHLT_Speech_corpus_zulu_50hr_v2",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_Fleurs_ZULU_63hr_v1",
# ]

In [ ]:
# models = [
#     "asr-africa/W2V2-Bert_NCHLT_Speech_corpus_ZULU_1hr_v1",
#     "asr-africa/W2V2-Bert_NCHLT_Speech_corpus_ZULU_5hr_v3",
#     "asr-africa/W2V2-Bert_NCHLT_Speech_corpus_ZULU_10hr_v1",
#     "asr-africa/W2V2-Bert_nchlt_speech_corpus_ZULU_20hr_v1",
#     "asr-africa/W2V2-Bert_nchlt_speech_corpus_ZULU_50hr_v1",
#     "asr-africa/W2V2-Bert_nchlt_speech_corpus_Fleurs_ZULU_63hr_v1",
# ]


In [ ]:
# models = [
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_1hr_v1",
#     "asr-africa/W2V2-Bert_NCHLT_Speech_corpus_ZULU_1hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_5hr_v1",
#     "asr-africa/W2V2-Bert_NCHLT_Speech_corpus_ZULU_5hr_v3",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_10hr_v1",
#     "asr-africa/W2V2-Bert_NCHLT_Speech_corpus_ZULU_10hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_ZULU_20hr_v1",
#     "asr-africa/W2V2-Bert_nchlt_speech_corpus_ZULU_20hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_NCHLT_Speech_corpus_zulu_50hr_v2",
#     "asr-africa/W2V2-Bert_nchlt_speech_corpus_ZULU_50hr_v1",
#     "asr-africa/wav2vec2_xls_r_300m_nchlt_speech_corpus_Fleurs_ZULU_63hr_v1",
#     "asr-africa/W2V2-Bert_nchlt_speech_corpus_Fleurs_ZULU_63hr_v1",
# ]


In [ ]:
from datasets import load_dataset, DatasetDict
import torch
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline
from tqdm import tqdm
import evaluate
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models=["asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-1hrs-2",
    "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-5hrs-2",
    "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-10hrs-3",
    "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-20hrs-3",
    "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-50hrs-3",
    "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-100hrs",
    "asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-200hrs-1",
    "asr-africa/w2v-bert-2.0-Amharic_Speech_corpusandamharic_dataset-amh-200hrs-2",
    "asr-africa/w2v-bert-2.0-amharic-dataset-amh-100hrs-2",
    "asr-africa/w2v-bert-2.0-amharic-dataset-amh-50hrs-2",
    "asr-africa/w2v-bert-2.0-amharic-dataset-amh-10hrs-2",
    "asr-africa/w2v-bert-2.0-amharic-dataset-amh-5hrs-2",
    "asr-africa/w2v-bert-2.0-amharic-dataset-amh-1hrs-2"
    # Add more model IDs as needed.
]

In [ ]:
from tqdm import tqdm
from transformers import pipeline
import pandas as pd
import time
import numpy as np
from requests.exceptions import ChunkedEncodingError
from huggingface_hub.utils import RepositoryNotFoundError
import evaluate  # Add missing import

# Initialize WER and CER metrics
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def load_model_with_retry(model_name, device):
    """Continuously retries downloading the model if the connection is interrupted."""
    while True:
        try:
            print(f"Loading model: {model_name}")
            pipe = pipeline("automatic-speech-recognition", model=model_name, device=device, return_timestamps='word')
            print(f"Successfully loaded model: {model_name}")
            return pipe  # Return the pipeline when successful
        except (ChunkedEncodingError, RepositoryNotFoundError) as e:
            print(f"Error loading model {model_name}: {e}. Retrying in 10 seconds...")
            time.sleep(10)  # Wait before retrying
        except Exception as e:
            print(f"Unexpected error: {e}. Retrying in 20 seconds...")
            time.sleep(20)

def ensure_audio_numpy(audio):
    """Ensure audio array is in the correct numpy format for ASR models."""
    if isinstance(audio, list):
        return np.array(audio)  # Convert list to numpy array
    elif isinstance(audio, dict) and "array" in audio:
        return np.array(audio["array"])  # Extract and convert array from dictionary
    elif isinstance(audio, np.ndarray):
        return audio  # Already in correct format
    else:
        raise ValueError(f"Unexpected audio format: {type(audio)}")

def run_inference_for_model(model_name, test5hr, device):
    """Runs ASR inference with error handling and retries for failed downloads."""

    pipe = load_model_with_retry(model_name, device)  # Load model with retry logic

    results = []
    all_predictions = []
    all_references = []
    total_substitutions, total_insertions, total_deletions, total_words = 0, 0, 0, 0

    for sample in tqdm(test5hr, total=len(test5hr), desc=f"Running inference for {model_name}"):
        try:
            audio = ensure_audio_numpy(sample["audio"])  # Ensure correct audio format
            sentence = sample["sentence"]

            # Run inference
            prediction = pipe(audio)['text']

            # Compute metrics
            wer, cer, errors = compute_individual_metrics(prediction, sentence)

            # Accumulate errors
            total_substitutions += errors["substitutions"]
            total_insertions += errors["insertions"]
            total_deletions += errors["deletions"]
            total_words += errors["total_words"]

            # Append results
            results.append({
                "Prediction": prediction,
                "Reference": sentence,
                "WER": wer,
                "CER": cer,
                "Substitutions": errors["substitutions"],
                "Insertions": errors["insertions"],
                "Deletions": errors["deletions"]
            })
            all_predictions.append(prediction)
            all_references.append(sentence)

        except Exception as e:
            print(f"Error processing audio sample: {e}. Skipping this sample.")

    # Create results DataFrame
    df = pd.DataFrame(results)

    # Compute overall metrics
    overall_wer = wer_metric.compute(predictions=all_predictions, references=all_references)
    overall_cer = cer_metric.compute(predictions=all_predictions, references=all_references)

    # Print overall metrics
    print(f"Model: {model_name}")
    print("Calculated overall WER:", overall_wer)
    print("Calculated overall CER:", overall_cer)
    print("Total Substitutions:", total_substitutions)
    print("Total Insertions:", total_insertions)
    print("Total Deletions:", total_deletions)
    print("Total Words:", total_words)
    print("Substitution Rate:", total_substitutions / total_words)
    print("Insertion Rate:", total_insertions / total_words)
    print("Deletion Rate:", total_deletions / total_words)

    # Save detailed results to CSV
    output_filename = f'sentences_{model_name.split("/")[-1]}.csv'
    df.to_csv(output_filename, index=False)
    print(f"Results saved to {output_filename}")

# Run inference for all models
for model in models:
    run_inference_for_model(model, test5hr, device)


In [ ]:
def compute_errors(prediction, reference):
    ref_words = reference.split()
    pred_words = prediction.split()

    # Initialize a matrix for Levenshtein alignment
    dp = [[0] * (len(pred_words) + 1) for _ in range(len(ref_words) + 1)]

    for i in range(len(ref_words) + 1):
        for j in range(len(pred_words) + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            else:
                if ref_words[i - 1] == pred_words[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1]
                else:
                    dp[i][j] = min(dp[i - 1][j - 1], dp[i - 1][j], dp[i][j - 1]) + 1

    # Backtrack to count substitutions, insertions, and deletions
    i, j = len(ref_words), len(pred_words)
    substitutions, insertions, deletions = 0, 0, 0

    while i > 0 and j > 0:
        if ref_words[i - 1] == pred_words[j - 1]:
            i -= 1
            j -= 1
        else:
            if dp[i][j] == dp[i - 1][j - 1] + 1:
                substitutions += 1
                i -= 1
                j -= 1
            elif dp[i][j] == dp[i - 1][j] + 1:
                deletions += 1
                i -= 1
            else:
                insertions += 1
                j -= 1

    while i > 0:
        deletions += 1
        i -= 1
    while j > 0:
        insertions += 1
        j -= 1

    return {

        "substitutions": substitutions,
        "insertions": insertions,
        "deletions": deletions,
        "total_words": len(ref_words),
    }

In [ ]:
def compute_individual_metrics(prediction, reference):
    wer = wer_metric.compute(predictions=[prediction], references=[reference])
    cer = cer_metric.compute(predictions=[prediction], references=[reference])
    errors = compute_errors(prediction, reference)
    return wer, cer, errors

In [ ]:
from tqdm import tqdm
from transformers import pipeline
import pandas as pd

def run_inference_for_model(model_name):
    print(f"Processing model: {model_name}")
    pipe = pipeline("automatic-speech-recognition", model=model_name, device=device, return_timestamps='word')
    results = []
    all_predictions = []
    all_references = []
    total_substitutions, total_insertions, total_deletions, total_words = 0, 0, 0, 0

    for audio, transcription in tqdm(zip(test5hr["audio"], test5hr["transcription"]),
                                     total=test5hr.num_rows, desc="Running inference"):
        # Run inference
        prediction = pipe(audio)['text']

        # Compute metrics
        wer, cer, errors = compute_individual_metrics(prediction, transcription)

        # Accumulate errors
        total_substitutions += errors["substitutions"]
        total_insertions += errors["insertions"]
        total_deletions += errors["deletions"]
        total_words += errors["total_words"]

        # Append results
        results.append({
            "Prediction": prediction,
            "Reference": transcription,
            "WER": wer,
            "CER": cer,
            "Substitutions": errors["substitutions"],
            "Insertions": errors["insertions"],
            "Deletions": errors["deletions"]
        })
        all_predictions.append(prediction)
        all_references.append(transcription)

    # Create results DataFrame
    df = pd.DataFrame(results)

    # Compute overall metrics
    overall_wer = wer_metric.compute(predictions=all_predictions, references=all_references)
    overall_cer = cer_metric.compute(predictions=all_predictions, references=all_references)

    # Print overall metrics
    print("Calculated overall WER:", overall_wer)
    print("Calculated overall CER:", overall_cer)
    print("Total Substitutions:", total_substitutions)
    print("Total Insertions:", total_insertions)
    print("Total Deletions:", total_deletions)
    print("Total Words:", total_words)
    print("Substitution Rate:", total_substitutions / total_words)
    print("Insertion Rate:", total_insertions / total_words)
    print("Deletion Rate:", total_deletions / total_words)

    # Save detailed results to CSV
    output_filename = f'transcriptions_{model_name.split("/")[-1]}.csv'
    df.to_csv(output_filename, index=False)
    print(f"Results saved to {output_filename}")


In [ ]:
for model in models:
    run_inference_for_model(model)

Processing model: asr-africa/wav2vec2-xls-r-300m-amharic-asr-dataset-50hrs-3


config.json:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

Running inference: 100%|██████████| 3163/3163 [10:49<00:00,  4.87it/s]


Calculated overall WER: 0.3199705489139912
Calculated overall CER: 0.1089415118995763
Total Substitutions: 13214
Total Insertions: 2137
Total Deletions: 1163
Total Words: 51611
Substitution Rate: 0.2560306911317355
Insertion Rate: 0.041405901842630446
Deletion Rate: 0.022533955939625275
Results saved to transcriptions_wav2vec2-xls-r-300m-amharic-asr-dataset-100hrs.csv


In [ ]:
from collections import Counter

# Function to find common errors
def find_common_errors(df, error_type="Substitutions"):
    errors = []
    for _, row in df.iterrows():
        ref_words = row["Reference"].split()
        pred_words = row["Prediction"].split()
        if error_type == "Substitutions":
            for ref, pred in zip(ref_words, pred_words):
                if ref != pred:
                    errors.append((ref, pred))
        elif error_type == "Insertions":
            if len(pred_words) > len(ref_words):
                for word in pred_words:
                    if word not in ref_words:
                        errors.append(word)
        elif error_type == "Deletions":
            if len(ref_words) > len(pred_words):
                for word in ref_words:
                    if word not in pred_words:
                        errors.append(word)
    return Counter(errors).most_common(10)

# Most common substitutions
print("Most Common Substitutions:")
print(find_common_errors(df, error_type="Substitutions"))

# Most common insertions
print("Most Common Insertions:")
print(find_common_errors(df, error_type="Insertions"))

# Most common deletions
print("Most Common Deletions:")
print(find_common_errors(df, error_type="Deletions"))

Most Common Substitutions:


NameError: name 'df' is not defined

In [ ]:
!pip install jiwer pandas

import pandas as pd
import numpy as np
from collections import defaultdict
import jiwer
import unicodedata

########################################
# 1. Helper: Normalize Amharic
########################################

def normalize_amharic(text):
    """
    Example normalization function for Amharic.
    Adjust as needed (remove or unify diacritics, punctuation, etc.).
    """
    return unicodedata.normalize("NFC", str(text))

########################################
# 2. Custom Alignment (Word-Level)
########################################

def align_tokens(ref_tokens, hyp_tokens):
    """
    Perform a simple dynamic programming alignment (Levenshtein).
    Returns a list of (ref_word, hyp_word, operation):
      operation ∈ {"hit", "sub", "ins", "del"}
    """
    n = len(ref_tokens)
    m = len(hyp_tokens)

    # dp[i][j] = (cost, alignment_list)
    dp = [[(0, []) for _ in range(m + 1)] for _ in range(n + 1)]

    # Initialize first column (all deletions)
    for i in range(1, n + 1):
        cost_prev, align_prev = dp[i - 1][0]
        dp[i][0] = (
            cost_prev + 1,
            align_prev + [(ref_tokens[i - 1], "", "del")]
        )

    # Initialize first row (all insertions)
    for j in range(1, m + 1):
        cost_prev, align_prev = dp[0][j - 1]
        dp[0][j] = (
            cost_prev + 1,
            align_prev + [("", hyp_tokens[j - 1], "ins")]
        )

    # Fill DP table
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            ref_word = ref_tokens[i - 1]
            hyp_word = hyp_tokens[j - 1]

            # If they match, no cost for this token
            if ref_word == hyp_word:
                cost_prev, align_prev = dp[i - 1][j - 1]
                dp[i][j] = (cost_prev, align_prev + [(ref_word, hyp_word, "hit")])
            else:
                # Substitution
                sub_cost, sub_align = dp[i - 1][j - 1]
                sub_cost += 1
                sub_align = sub_align + [(ref_word, hyp_word, "sub")]

                # Insertion
                ins_cost, ins_align = dp[i][j - 1]
                ins_cost += 1
                ins_align = ins_align + [("", hyp_word, "ins")]

                # Deletion
                del_cost, del_align = dp[i - 1][j]
                del_cost += 1
                del_align = del_align + [(ref_word, "", "del")]

                # Choose minimal cost
                if sub_cost <= ins_cost and sub_cost <= del_cost:
                    dp[i][j] = (sub_cost, sub_align)
                elif ins_cost <= del_cost:
                    dp[i][j] = (ins_cost, ins_align)
                else:
                    dp[i][j] = (del_cost, del_align)

    # Final alignment is in dp[n][m]
    _, alignment_list = dp[n][m]
    return alignment_list

########################################
# 3. Metrics + Alignment
########################################

def compute_individual_metrics(hypothesis, reference):
    """
    Compute WER, CER, error counts, plus alignment.
    Returns:
      wer_val, cer_val, errors, alignment
    """
    ref_norm = normalize_amharic(reference)
    hyp_norm = normalize_amharic(hypothesis)

    # Use jiwer for WER, CER
    measures = jiwer.compute_measures(ref_norm, hyp_norm)
    wer_val  = measures.get("wer", 0.0)
    sub      = measures.get("substitutions", 0)
    ins      = measures.get("insertions", 0)
    dele     = measures.get("deletions", 0)
    total_words = len(ref_norm.split())
    errors = {
        "substitutions": sub,
        "insertions": ins,
        "deletions": dele,
        "total_words": total_words
    }

    cer_val = jiwer.cer(ref_norm, hyp_norm)

    # Word-level alignment
    ref_tokens = ref_norm.split()
    hyp_tokens = hyp_norm.split()
    alignment  = align_tokens(ref_tokens, hyp_tokens)

    return wer_val, cer_val, errors, alignment

########################################
# 4. Optional: Character-Level Difference
########################################

def compare_characters(ref_word, hyp_word):
    """
    Returns a list of (ref_char, hyp_char) pairs
    for each position. If lengths differ, extra chars
    are paired with ''.
    """
    max_len = max(len(ref_word), len(hyp_word))
    pairs = []
    for i in range(max_len):
        r_char = ref_word[i] if i < len(ref_word) else ""
        h_char = hyp_word[i] if i < len(hyp_word) else ""
        if r_char != h_char:
            pairs.append((r_char, h_char))
    return pairs

########################################
# 5. Analyze Substitution Context
########################################

def analyze_substitution_contexts(substitution_list, top_n=10):
    """
    - substitution_list: list of dict with keys:
        {
          "preceding_context": list[str],
          "ref_word": str,
          "hyp_word": str,
          "next_word": str or None
        }
    - top_n: how many top patterns to print
    """
    # (ref_word -> hyp_word) counts
    sub_counts = defaultdict(int)
    # preceding context counts
    context_counts = defaultdict(int)

    for item in substitution_list:
        ref_word  = item["ref_word"]
        hyp_word  = item["hyp_word"]
        ctx_tuple = tuple(item["preceding_context"])
        sub_counts[(ref_word, hyp_word)] += 1
        context_counts[ctx_tuple]        += 1

    # Show top N frequent substitutions
    sorted_sub_counts = sorted(sub_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Substitution Word Patterns:")
    for (ref, hyp), count in sorted_sub_counts[:top_n]:
        print(f"  '{ref}' -> '{hyp}': {count} times")

    # Show top N preceding contexts
    sorted_ctx_counts = sorted(context_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Preceding Contexts Causing Substitutions:")
    for ctx, count in sorted_ctx_counts[:top_n]:
        print(f"  Context {ctx}: {count} times")

########################################
# 6. Main Analysis Function
########################################

def analyze_csv(
    csv_file,
    output_analyzed="results_analyzed.csv",
    output_sub_contexts="substitution_contexts.csv",
    do_char_diff=False
):
    """
    - csv_file: path to CSV with columns ["Prediction", "Reference"].
    - output_analyzed: row-level metrics output CSV
    - output_sub_contexts: bigram/trigram context output CSV
    - do_char_diff: if True, store character-level diffs for each substitution

    Steps:
    1) read CSV
    2) for each row => compute WER, CER, alignment
    3) gather substitution contexts
    4) write row-level results to <output_analyzed>
    5) analyze + write aggregated contexts to <output_sub_contexts>
    """
    df = pd.read_csv(csv_file)
    results_rows = []
    substitution_list = []

    total_subs = 0
    total_ins  = 0
    total_del  = 0
    total_wds  = 0

    # Process each row
    for idx, row in df.iterrows():
        pred = str(row["Prediction"])
        ref  = str(row["Reference"])

        wer_val, cer_val, errors, alignment = compute_individual_metrics(pred, ref)
        total_subs += errors["substitutions"]
        total_ins  += errors["insertions"]
        total_del  += errors["deletions"]
        total_wds  += errors["total_words"]

        # gather bigram/trigram context for each sub
        ref_tokens = [a[0] for a in alignment]  # after alignment
        hyp_tokens = [a[1] for a in alignment]
        ops        = [a[2] for a in alignment]

        # For each alignment step, if 'sub', collect context
        # We'll do 2 preceding words + the next word (trigram).
        for i, op_type in enumerate(ops):
            if op_type == "sub":
                start_idx = max(0, i - 2)   # up to 2 preceding
                preceding_context = [t for t in ref_tokens[start_idx:i] if t]

                ref_word = ref_tokens[i]
                hyp_word = hyp_tokens[i]
                next_word = ref_tokens[i + 1] if (i + 1) < len(ref_tokens) else None

                # optional character-level analysis
                char_diffs = compare_characters(ref_word, hyp_word) if do_char_diff else []

                substitution_list.append({
                    "row_idx": idx,
                    "preceding_context": preceding_context,
                    "ref_word": ref_word,
                    "hyp_word": hyp_word,
                    "next_word": next_word,
                    "char_diffs": char_diffs
                })

        results_rows.append({
            "Prediction": pred,
            "Reference": ref,
            "WER": wer_val,
            "CER": cer_val,
            "Substitutions": errors["substitutions"],
            "Insertions": errors["insertions"],
            "Deletions": errors["deletions"]
        })

    results_df = pd.DataFrame(results_rows)

    # Overall metrics
    overall_wer = jiwer.wer(list(df["Reference"]), list(df["Prediction"]))
    overall_cer = jiwer.cer(list(df["Reference"]), list(df["Prediction"]))

    print("\n=== Overall Metrics ===")
    print(f"Calculated overall WER: {overall_wer}")
    print(f"Calculated overall CER: {overall_cer}")
    print(f"Total Substitutions: {total_subs}")
    print(f"Total Insertions: {total_ins}")
    print(f"Total Deletions: {total_del}")
    print(f"Total Words: {total_wds}")
    if total_wds > 0:
        print(f"Substitution Rate: {total_subs / total_wds}")
        print(f"Insertion Rate: {total_ins / total_wds}")
        print(f"Deletion Rate: {total_del / total_wds}")

    results_df.to_csv(output_analyzed, index=False)
    print(f"\nRow-level results saved to: {output_analyzed}")

    # Now let's do aggregated substitution analysis
    analyze_substitution_contexts(substitution_list, top_n=10)

    # Optionally write out the list of all substitution contexts
    sub_df = pd.DataFrame(substitution_list)
    sub_df.to_csv(output_sub_contexts, index=False)
    print(f"Detailed substitution contexts saved to: {output_sub_contexts}")

##################################
# Example usage in Colab
##################################
csv_path = "transcriptions_wav2vec2-xls-r-300m-amharic-asr-dataset-100hrs.csv"

# Turn on do_char_diff if you want per-substitution character analysis
analyze_csv(csv_file=csv_path,
            output_analyzed="my_amharic_analyzed.csv",
            output_sub_contexts="my_subcontexts.csv",
            do_char_diff=True)



=== Overall Metrics ===
Calculated overall WER: 0.3199705489139912
Calculated overall CER: 0.11002653150120778
Total Substitutions: 13098
Total Insertions: 2195
Total Deletions: 1221
Total Words: 51611
Substitution Rate: 0.2537831082521168
Insertion Rate: 0.04252969328243979
Deletion Rate: 0.023657747379434615

Row-level results saved to: my_amharic_analyzed.csv

Top 10 Substitution Word Patterns:
  'እና' -> 'ና': 74 times
  'እና' -> 'እ': 27 times
  'እናም' -> 'እና': 14 times
  'ወይም' -> 'ወም': 11 times
  'ሚሊዮን' -> 'ሚሊየን': 9 times
  'ወይም' -> 'ም': 9 times
  'አለብዎት' -> 'አለቦት': 8 times
  'ይችላል' -> 'ይችላሉ': 8 times
  'ሀብረት' -> 'ህብረት': 8 times
  'ሰዎችን' -> 'ሰዎች': 8 times

Top 10 Preceding Contexts Causing Substitutions:
  Context (): 1104 times
  Context ('በ',): 61 times
  Context ('እና',): 41 times
  Context ('ውስጥ',): 25 times
  Context ('ከ',): 22 times
  Context ('የ',): 17 times
  Context ('ነገር', 'ግን'): 16 times
  Context ('ይህ',): 16 times
  Context ('ወደ',): 16 times
  Context ('ብዙውን', 'ጊዜ'): 15 ti

In [ ]:
!pip install jiwer pandas

import pandas as pd
import numpy as np
from collections import defaultdict
import jiwer
import unicodedata

########################################
# 1. Helper: Normalize Amharic
########################################

def normalize_amharic(text):
    """
    Example normalization function for Amharic.
    Adjust as needed (remove or unify diacritics, punctuation, etc.).
    """
    return unicodedata.normalize("NFC", str(text))

########################################
# 2. Custom Alignment (Word-Level)
########################################

def align_tokens(ref_tokens, hyp_tokens):
    """
    Perform a simple dynamic programming alignment (Levenshtein).
    Returns a list of (ref_word, hyp_word, operation):
      operation ∈ {"hit", "sub", "ins", "del"}
    """
    n = len(ref_tokens)
    m = len(hyp_tokens)

    # dp[i][j] = (cost, alignment_list)
    dp = [[(0, []) for _ in range(m + 1)] for _ in range(n + 1)]

    # Initialize first column (all deletions)
    for i in range(1, n + 1):
        cost_prev, align_prev = dp[i - 1][0]
        dp[i][0] = (
            cost_prev + 1,
            align_prev + [(ref_tokens[i - 1], "", "del")]
        )

    # Initialize first row (all insertions)
    for j in range(1, m + 1):
        cost_prev, align_prev = dp[0][j - 1]
        dp[0][j] = (
            cost_prev + 1,
            align_prev + [("", hyp_tokens[j - 1], "ins")]
        )

    # Fill DP table
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            ref_word = ref_tokens[i - 1]
            hyp_word = hyp_tokens[j - 1]

            # If they match, no cost for this token
            if ref_word == hyp_word:
                cost_prev, align_prev = dp[i - 1][j - 1]
                dp[i][j] = (cost_prev, align_prev + [(ref_word, hyp_word, "hit")])
            else:
                # Substitution
                sub_cost, sub_align = dp[i - 1][j - 1]
                sub_cost += 1
                sub_align = sub_align + [(ref_word, hyp_word, "sub")]

                # Insertion
                ins_cost, ins_align = dp[i][j - 1]
                ins_cost += 1
                ins_align = ins_align + [("", hyp_word, "ins")]

                # Deletion
                del_cost, del_align = dp[i - 1][j]
                del_cost += 1
                del_align = del_align + [(ref_word, "", "del")]

                # Choose minimal cost
                if sub_cost <= ins_cost and sub_cost <= del_cost:
                    dp[i][j] = (sub_cost, sub_align)
                elif ins_cost <= del_cost:
                    dp[i][j] = (ins_cost, ins_align)
                else:
                    dp[i][j] = (del_cost, del_align)

    # Final alignment is in dp[n][m]
    _, alignment_list = dp[n][m]
    return alignment_list

########################################
# 3. Levenshtein Edit Distance (Char-Level)
########################################

def levenshtein_distance(a, b):
    """
    Simple DP approach to compute edit distance between two strings a, b.
    """
    A, B = len(a), len(b)
    dp = np.zeros((A + 1, B + 1), dtype=int)
    for i in range(A + 1):
        dp[i][0] = i
    for j in range(B + 1):
        dp[0][j] = j

    for i in range(1, A + 1):
        for j in range(1, B + 1):
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[i][j] = min(
                dp[i-1][j] + 1,     # deletion
                dp[i][j-1] + 1,     # insertion
                dp[i-1][j-1] + cost # substitution
            )
    return dp[A][B]

########################################
# 4. Morphological "Root" (Placeholder)
########################################

def get_morphological_root(amharic_word):
    """
    Naive placeholder for morphological analysis.
    Removes common suffixes, etc.
    You can integrate a real morphological analyzer here.

    Example suffixes (very naive):
      - ም, ማ, ች, ቹ, ው, ዎች, ነ, etc.

    NOTE: This is a simplistic example.
    Customize or replace with a real parser for better results.
    """
    # Common suffixes you want to strip out
    suffixes = ["ም", "ማ", "ች", "ቹ", "ው", "ዎች", "ነ", "ዋ", "ቷ", "ሗ", "ሉ"]
    root = amharic_word
    for sfx in suffixes:
        if root.endswith(sfx) and len(root) > len(sfx):
            root = root[: -len(sfx)]
    return root

########################################
# 5. Main Metrics + Substitution Extraction
########################################

def compute_individual_metrics(hypothesis, reference):
    """
    Compute WER, CER, error counts, plus alignment.
    Returns:
      wer_val, cer_val, errors, alignment
    """
    ref_norm = normalize_amharic(reference)
    hyp_norm = normalize_amharic(hypothesis)

    # Use jiwer for WER, CER
    measures = jiwer.compute_measures(ref_norm, hyp_norm)
    wer_val  = measures.get("wer", 0.0)
    sub      = measures.get("substitutions", 0)
    ins      = measures.get("insertions", 0)
    dele     = measures.get("deletions", 0)
    total_words = len(ref_norm.split())
    errors = {
        "substitutions": sub,
        "insertions": ins,
        "deletions": dele,
        "total_words": total_words
    }

    cer_val = jiwer.cer(ref_norm, hyp_norm)

    # Word-level alignment
    ref_tokens = ref_norm.split()
    hyp_tokens = hyp_norm.split()
    alignment  = align_tokens(ref_tokens, hyp_tokens)

    return wer_val, cer_val, errors, alignment

########################################
# 6. Comprehensive Substitution Analysis
########################################

def analyze_substitutions_deeper(substitution_list, top_n=10):
    """
    For each item in substitution_list (which has keys like):
      {
        "preceding_context": [...],
        "ref_word": str,
        "hyp_word": str,
        "char_distance": int (Levenshtein distance),
        "ref_root": str,
        "hyp_root": str
      }
    we do:

      1. Print top-n frequent (ref_word -> hyp_word) pairs
      2. Print top-n frequent preceding contexts
      3. Print top-n frequent root pairs
      4. Print top-n frequent character mismatches across the entire dataset
         (which glyph is replaced by which glyph)
    """

    # 1) Count (ref_word -> hyp_word)
    pair_counts = defaultdict(int)
    # 2) preceding context
    context_counts = defaultdict(int)
    # 3) (ref_root -> hyp_root)
    root_counts = defaultdict(int)
    # 4) track all character pairs (char -> char)
    char_pair_counts = defaultdict(int)

    for item in substitution_list:
        r_w = item["ref_word"]
        h_w = item["hyp_word"]
        ctx_tuple = tuple(item["preceding_context"])
        r_root = item["ref_root"]
        h_root = item["hyp_root"]
        # gather char-level mismatches
        for (r_char, h_char) in item["char_mismatches"]:
            char_pair_counts[(r_char, h_char)] += 1

        pair_counts[(r_w, h_w)] += 1
        context_counts[ctx_tuple] += 1
        root_counts[(r_root, h_root)] += 1

    # Print top-n frequent (ref->hyp)
    sorted_pair_counts = sorted(pair_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Substitution Word Patterns:")
    for (r, h), cnt in sorted_pair_counts[:top_n]:
        print(f"  '{r}' -> '{h}': {cnt} times")

    # Print top-n preceding contexts
    sorted_ctx_counts = sorted(context_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Preceding Contexts Causing Substitutions:")
    for ctx, cnt in sorted_ctx_counts[:top_n]:
        print(f"  Context {ctx}: {cnt} times")

    # Print top-n root pairs
    sorted_root_counts = sorted(root_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Substitution Root Patterns:")
    for (r_root, h_root), cnt in sorted_root_counts[:top_n]:
        # skip trivial empty roots if any
        if not r_root and not h_root:
            continue
        print(f"  '{r_root}' -> '{h_root}': {cnt} times")

    # Print top-n character mismatch pairs
    sorted_char_pairs = sorted(char_pair_counts.items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop {top_n} Character Mismatch Pairs (Across All Subs):")
    for (r_c, h_c), cnt in sorted_char_pairs[:top_n]:
        # show them as 'እ'->'ኧ' : 12 times
        print(f"  '{r_c}' -> '{h_c}': {cnt} times")

########################################
# 7. Main "analyze_csv" function
########################################

def analyze_csv(csv_file, top_n=10):
    """
    - csv_file: path to CSV with columns ["Prediction", "Reference"].
    - We do a deeper analysis:
      (a) WER/CER row-level
      (b) Substitution context extraction (with bigram preceding words)
      (c) Morphological "root" + character-level edit distance
      (d) Summaries all displayed in console (no extra CSV).
    """
    df = pd.read_csv(csv_file)
    results_rows = []
    substitution_list = []

    total_subs = 0
    total_ins  = 0
    total_del  = 0
    total_wds  = 0

    # Process each row
    for idx, row in df.iterrows():
        pred = str(row["Prediction"])
        ref  = str(row["Reference"])

        wer_val, cer_val, errors, alignment = compute_individual_metrics(pred, ref)
        total_subs += errors["substitutions"]
        total_ins  += errors["insertions"]
        total_del  += errors["deletions"]
        total_wds  += errors["total_words"]

        # gather bigram/trigram context for each sub
        ref_tokens = [a[0] for a in alignment]
        hyp_tokens = [a[1] for a in alignment]
        ops        = [a[2] for a in alignment]

        for i, op_type in enumerate(ops):
            if op_type == "sub":
                start_idx = max(0, i - 2)
                preceding_context = [t for t in ref_tokens[start_idx:i] if t]

                ref_word = ref_tokens[i]
                hyp_word = hyp_tokens[i]
                next_word = ref_tokens[i + 1] if (i + 1) < len(ref_tokens) else None

                # Compute character-level edit distance & mismatches
                char_dist = levenshtein_distance(ref_word, hyp_word)
                # figure out character mismatches quickly
                # naive approach: pair up each position, ignoring alignment
                # or do a quick approach:
                max_len = max(len(ref_word), len(hyp_word))
                char_mismatches = []
                for c_idx in range(max_len):
                    r_c = ref_word[c_idx] if c_idx < len(ref_word) else ""
                    h_c = hyp_word[c_idx] if c_idx < len(hyp_word) else ""
                    if r_c != h_c:
                        char_mismatches.append((r_c, h_c))

                # morphological root extraction
                ref_root = get_morphological_root(ref_word)
                hyp_root = get_morphological_root(hyp_word)

                substitution_list.append({
                    "row_idx": idx,
                    "preceding_context": preceding_context,
                    "ref_word": ref_word,
                    "hyp_word": hyp_word,
                    "next_word": next_word,
                    "char_distance": char_dist,
                    "char_mismatches": char_mismatches,
                    "ref_root": ref_root,
                    "hyp_root": hyp_root
                })

        results_rows.append({
            "Prediction": pred,
            "Reference": ref,
            "WER": wer_val,
            "CER": cer_val,
            "Substitutions": errors["substitutions"],
            "Insertions": errors["insertions"],
            "Deletions": errors["deletions"]
        })

    results_df = pd.DataFrame(results_rows)

    # Overall metrics
    # We'll also do a final pass with jiwer to confirm
    overall_wer = jiwer.wer(list(df["Reference"]), list(df["Prediction"]))
    overall_cer = jiwer.cer(list(df["Reference"]), list(df["Prediction"]))

    print("\n=== Overall Metrics ===")
    print(f"Calculated overall WER: {overall_wer}")
    print(f"Calculated overall CER: {overall_cer}")
    print(f"Total Substitutions: {total_subs}")
    print(f"Total Insertions: {total_ins}")
    print(f"Total Deletions: {total_del}")
    print(f"Total Words: {total_wds}")
    if total_wds > 0:
        print(f"Substitution Rate: {total_subs / total_wds}")
        print(f"Insertion Rate: {total_ins / total_wds}")
        print(f"Deletion Rate: {total_del / total_wds}")

    # Print row-level summary (first few rows) if desired
    print("\n=== Sample of Row-Level Results ===")
    print(results_df.head(5))

    # Deeper analysis: morphological root clustering, char-level mismatch stats, etc.
    analyze_substitutions_deeper(substitution_list, top_n=top_n)

    # Show sample from substitution_list
    print("\n=== Sample of Substitution Contexts (mysubcontexts) ===")
    for item in substitution_list[:top_n]:
        print(item)

########################################
# 8. Example usage
########################################
csv_path = "transcriptions_wav2vec2-xls-r-300m-amharic-asr-dataset-100hrs.csv"
analyze_csv(csv_file=csv_path, top_n=10)



=== Overall Metrics ===
Calculated overall WER: 0.3199705489139912
Calculated overall CER: 0.11002653150120778
Total Substitutions: 13098
Total Insertions: 2195
Total Deletions: 1221
Total Words: 51611
Substitution Rate: 0.2537831082521168
Insertion Rate: 0.04252969328243979
Deletion Rate: 0.023657747379434615

=== Sample of Row-Level Results ===
                                          Prediction  \
0  ምግብ ከሱፐርማርኬት መግዛት በተለምዶ ርካሹ ራስን የመመገበ ዘዴ ነው ምግ...   
1  ጃስሰይ ሪፐን በማኮብፖበ ሜዳ ላይ በኝ ተኩል ኤኤም አከባቢያዊ ሰአት በዜ...   
2  ሰው አልባው የጨረቃ ተላይትቻንድሪያን አንድ የሞ ኢንፓክት ፒሮቡን ወይም ...   
3  ቅፅበታዊ የፅሁፍ አስተርጓሚ መተግበሪያዎች ሙሉ የፅሁፍ ክፍሎችን ከአንድ ...   
4     ሶውዬት ዩኒን ትልቅ ክብር የሶቬት ዩኒየን ጀግና ለስራው ስራ ሁሉ ተሰጠው   

                                           Reference       WER       CER  \
0  ምግብ ከሱፐርማርኬት መግዛት በተለምዶ ርካሹ እራስን የመመገቢያ ዘዴ ነው ...  0.238095  0.057692   
1  በማኮብኮብያ ሜዳ ላይ በ አካባቢያዊ ሰእት ተከሰከሰ እና ፈነዳ ስለሆነም ...  0.764706  0.506329   
2  ሰው አልባው የጨረቃ ሳተላይት ቻንድራያን የሙን ኢምፓክት ፕሮቡን ኤምኢይፒ...  0.640000  0.303279   
3  ቅፅበታዊ 